# Creating Fresh Data Base

In [7]:
import psycopg2

def get_create_table_queries(conn):
    create_table_queries = []
    try:
        # Create a cursor object
        cur = conn.cursor()

        # Get the list of all schemas excluding temporary and system schemas
        schema_sql = """
        SELECT schema_name 
        FROM information_schema.schemata
        WHERE schema_name NOT LIKE 'pg_temp_%'
        AND schema_name NOT LIKE 'pg_toast%'
        AND schema_name NOT LIKE 'information%'
        AND schema_name NOT LIKE 'pg_catalog%'
        """
        cur.execute(schema_sql)
        schemas = cur.fetchall()

        # Get the CREATE TABLE statements for all tables in each schema
        for schema in schemas:
            schema_name = schema[0]
            table_sql = f"""
            SELECT table_name 
            FROM information_schema.tables
            WHERE table_schema = '{schema_name}'
            """
            cur.execute(table_sql)
            tables = cur.fetchall()

            for table in tables:
                table_name = table[0]
                create_sql = f"""
                SELECT 'CREATE TABLE ' || quote_ident('{schema_name}') || '.' || quote_ident('{table_name}') || E'\n(\n' ||
                array_to_string(
                    array_agg(
                        '    ' || quote_ident(column_name) || ' ' || data_type ||
                        coalesce('(' || character_maximum_length || ')', '') ||
                        case when is_nullable = 'NO' then ' NOT NULL' else '' end
                    ), E',\n'
                ) || E'\n);\n'
                FROM information_schema.columns
                WHERE table_schema = '{schema_name}' AND table_name = '{table_name}'
                GROUP BY table_schema, table_name
                """
                cur.execute(create_sql)
                create_table_query = cur.fetchone()[0]
                create_table_queries.append(create_table_query)

        # Close the cursor
        cur.close()

    except Exception as e:
        print(f"Error: {e}")

    return create_table_queries

def save_to_sql_file(filename, queries):
    with open(filename, 'w') as f:
        for query in queries:
            f.write(query + '\n\n')


conn = psycopg2.connect(
    host="192.168.2.54",
    port=5432,
    database="BravisaDB",
    user="postgres",
    password="edsols"
)

# create the schemas


create_table_queries = get_create_table_queries(conn)
save_to_sql_file("create_tables.sql", create_table_queries)


# Close the connection
conn.close()

# Run this cell to create all the tables and schemas

In [2]:
import psycopg2
# connect to local host db named BravisaDB
conn = psycopg2.connect(
    host="localhost",
    port=5432,
    database="BravisaDB",
    user="postgres",
    password="edsols"
)

# run the create_table.sql file 
with open("create_tables.sql", 'r') as f:
    create_table_sql = f.read()
    cur = conn.cursor()
    cur.execute(create_table_sql)
    conn.commit()
    cur.close()
    

In [17]:
import os

cwd = os.getcwd()

ohlc = os.path.join(cwd, "app\\OHLCFiles")

# for files starting with cm and ending with .csv rename then to upper()
for file in os.listdir(ohlc):
    if file.startswith("cm") and file.endswith(".csv"):
        os.rename(os.path.join(ohlc, file), os.path.join(ohlc, file.upper()))

In [ ]:
import datetime
import os
import pandas as pd

cwd = os.getcwd()

ohlc = os.path.join(cwd, "app\\OHLCFiles\\")
indexfiles = os.path.join(cwd, "app\\IndexOHLCFiles\\")

# pandas date_range
date_range = pd.date_range(start='01-01-2020', end='18-10-2024', freq='D')

for curr_date in date_range:
    # Format the date for NSE file naming convention
    download_date_nse = curr_date.strftime("%d%b%Y").upper()
    file_nse = f'CM{download_date_nse}BHAV.CSV'

    # Format the date for EQ_ISINCODE file naming convention
    download_date_bse = curr_date.strftime("%d%m%y")
    file_bse = f"EQ_ISINCODE_{download_date_bse}.CSV"

    ohlc_files = os.listdir(ohlc)


    #if file_nse and file_bse are both not in the ohlc_files list
    # if file_nse not in ohlc_files and file_bse not in ohlc_files:
    #     # ignore saturday and sunday
    #     if(curr_date.weekday() == 5 or curr_date.weekday() == 6):
    #         continue
    #     print(f"Both {file_nse} and {file_bse} are not present in the directory")
    # else:
    #     if file_nse not in ohlc_files:
    #         print(f"{file_nse} is not present in the directory")

    #     if file_bse not in ohlc_files:
    #         print(f"{file_bse} is not present in the directory")

    # ind_close_all_01012020.csv check for this file in the IndexOHLCFiles directory
    download_date_index = curr_date.strftime("%d%m%Y")
    file_index = f"ind_close_all_{download_date_index}.csv"

    index_files = os.listdir(indexfiles)

    if file_index not in index_files:
        if(curr_date.weekday() == 5 or curr_date.weekday() == 6):
            continue
        print(f"{file_index} is not present in the directory")
    



In [ ]:
import psycopg2
from collections import Counter
# Expected data types for each column by table name
expected_data_types = {
    "FRS-NAVCategoryAvg": {
    "btt_scheme_category": str,
    "Date": pd.to_datetime,
    "1 Day Average": float,
    "1 Week Average": float,
    "1 Month Average": float,
    "3 Month Average": float,
    "6 Month Average": float,
    "9 Month Average": float,
    "1 Year Average": float,
    "2 Year Average": float,
    "3 Year Average": float,
    "5 Year Average": float,
    },
"FRS-MFRank":    {
    "CompanyCode": float,
    "CompanyName": str,
    "Date": pd.to_datetime,
    "ISINCode": str,
    "Quantity": float,
    "OutstandingShares": float,
    "MFExposure": float,
    "MFRank": float,
},
"CombinedRS" : {
    "CompanyCode": float,
    "CompanyName": str,
    "NSECode": str,
    "BSECode": int,
    "PRSRank": float,
    "ERSRank": float,
    "RRSRank": float,
    "FRSRank": float,
    "IRSRank": float,
    "CombiRS": float,
    "Rank": int,
    "Value Average": float,
    "GenDate": pd.to_datetime,
},
"AnnualGeneralMeeting": {
    "CompanyCode": float,
    "DateOfAnnouncement": pd.to_datetime,
    "AGMDate": pd.to_datetime,
    "YearEnding": pd.to_datetime,
    "Purpose": str,
    "RecordDate": pd.to_datetime,
    "BookClosureStartDate": pd.to_datetime,
    "BookClosureEndDate": pd.to_datetime,
    "Remarks": str,
    "DeleteFlag": bool,
    "ModifiedDate": pd.to_datetime,
},
"Auditors": {
    "Companycode": float,
    "AgencyName": str,
    "ModifiedDate": pd.to_datetime,
},
"BSE": {
    "SC_CODE": float,
    "SC_NAME": str,
    "SC_GROUP": str,
    "SC_TYPE": str,
    "OPEN": float,
    "HIGH": float,
    "LOW": float,
    "CLOSE": float,
    "LAST": float,
    "PREVCLOSE": float,
    "NO_TRADES": float,
    "NO_OF_SHRS": float,
    "NET_TURNOV": float,
    "TDCLOINDI": str,
    "ISIN_CODE": str,
    "TRADING_DATE": pd.to_datetime,
},
"BTTDivisor": {
    "IndexName": str,
    "BTTIndexValue": float,
    "Divisor": float,
    "Date": pd.to_datetime,
    "OS": int,
},
"BTTDivisor_DUPLICATED": {
    "IndexName": str,
    "BTTIndexValue": float,
    "Divisor": float,
    "Date": pd.to_datetime,
    "OS": int,
},
"BTTList": {
    "CompanyName": str,
    "ISIN": str,
    "NSECode": str,
    "BSECode": float,
    "Date": pd.to_datetime,
    "MFList": bool,
    "BTTDate": pd.to_datetime,
    "CompanyCode": float,
},
"BTTListCopy": {
    "CompanyName": str,
    "ISIN": str,
    "NSECode": str,
    "BSECode": int,
    "Date": pd.to_datetime,
    "MFList": bool,
    "BTTDate": pd.to_datetime,
    "CompanyCode": float,
},
"BTTList_DUPLICATED": {
    "CompanyName": str,
    "ISIN": str,
    "NSECode": str,
    "BSECode": int,
    "Date": pd.to_datetime,
    "MFList": bool,
    "BTTDate": pd.to_datetime,
    "CompanyCode": float,
    "Updated": str,
},
"BTTList_STABLE": {
    "CompanyName": str,
    "ISIN": str,
    "NSECode": str,
    "BSECode": float,
    "Date": pd.to_datetime,
    "MFList": bool,
    "BTTDate": pd.to_datetime,
    "CompanyCode": float,
},
"BackgroundInfo": {
    "CompanyCode": float,
    "CompanyName": str,
    "ShortCompanyName": str,
    "TickerName": str,
    "ParentCompanyCode": float,
    "PersonDescription": str,
    "PersonName": str,
    "ExecutiveDescription": str,
    "ExecutiveName": str,
    "BSECode": int,
    "BSEGroup": str,
    "NSECode": str,
    "ISINCode": str,
    "Sourcetype": str,
    "IncorporationDate": pd.to_datetime,
    "FirstPublicIssueDate": pd.to_datetime,
    "Auditeddate": pd.to_datetime,
    "FaceValue": float,
    "Marketlot": float,
    "IndustryCode": int,
    "IndustryName": str,
    "BusinessGroupCode": int,
    "BusinessGroupName": str,
    "MajorSector": int,
    "latestFinYear": str,
    "ForemostFinYear": str,
    "LatestHalfYear": str,
    "ForemostHalfYear": str,
    "LatestQtrYear": str,
    "ForemostQtrYear": str,
    "ModifiedDate": pd.to_datetime,
},
"Bankers": {
    "Companycode": float,
    "AgencyName": str,
    "ModifiedDate": pd.to_datetime,
},
"Blockdeals": {
    "CompanyCode": float,
    "Exchange": str,
    "Symbol": str,
    "DealDate": pd.to_datetime,
    "DealType": str,
    "ClientName": str,
    "Quantity": int,
    "Price": float,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
},
"BoardMeetings": {
    "CompanyCode": float,
    "BoardMeetDate": pd.to_datetime,
    "Purpose": str,
    "Remarks": str,
    "DeleteFlag": bool,
    "ModifiedDate": pd.to_datetime,
},
"Bonus": {
    "CompanyCode": float,
    "DateOfAnnouncement": pd.to_datetime,
    "RatioExisting": float,
    "RatioOfferred": float,
    "RecordDate": pd.to_datetime,
    "BookClosureStartDate": pd.to_datetime,
    "BookClosureEndDate": pd.to_datetime,
    "XBDate": pd.to_datetime,
    "Remarks": str,
    "DeleteFlag": bool,
    "ModifiedDate": pd.to_datetime,
},
"BulkDeals": {
    "CompanyCode": float,
    "Exchange": str,
    "Symbol": str,
    "DealDate": pd.to_datetime,
    "DealType": str,
    "ClientName": str,
    "Quantity": int,
    "Price": float,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
},
"CapitalStructure": {
    "companycode": float,
    "FromYear": float,
    "ToYear": float,
    "InstrumentName": str,
    "ReferencePercentage": float,
    "AuthorizedShares": float,
    "AuthorizedFaceValue": float,
    "AuthorizedCapital": float,
    "IssuedShares": float,
    "IssuedCapital": float,
    "NoOfPaidUpShares": float,
    "PaidUpFaceValue": float,
    "PaidUpCapital": float,
    "TotalArrears": float,
    "Reasonsforchange": str,
    "ModifiedDate": pd.to_datetime,
},
"CompanyMaster": {
    "CompanyCode": float,
    "CompanyName": str,
    "ShortCompanyName": str,
    "IndustryCode": int,
    "IndustryName": str,
    "BusinessGroupCode": int,
    "BusinessGroupName": str,
    "IncorporationDate": pd.to_datetime,
    "IncorporationYear": str,
    "FirstPublicIssueDate": pd.to_datetime,
    "FirstPublicIssueYear": str,
    "MajorSector": int,
    "CINNo": str,
    "Remarks": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag ": bool,
},
"CompanyNameChange": {
    "CompanyCode": float,
    "EffectiveDate": pd.to_datetime,
    "OldCompanyName": str,
    "NewCompanyName": str,
    "Remarks": str,
    "ModifiedDate": pd.to_datetime,
},
"ConsolidatedHalfYearlyResults": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Half": int,
    "OperatingIncome": float,
    "OtherOperatingIncome": float,
    "TotalIncomeFromOperations": float,
    "IntOrDiscOnAdvOrBills": float,
    "IncomeOnInvestements": float,
    "IntOnBalancesWithRBI": float,
    "Others": float,
    "OtherRecurringIncome": float,
    "StockAdjustment": float,
    "RawMaterialConsumed": float,
    "PurchaseOfTradedGoods": float,
    "PowerAndFuel": float,
    "EmployeeExpenses": float,
    "Excise": float,
    "AdminAndSellingExpenses": float,
    "ResearchAndDevelopmentExpenses": float,
    "ExpensesCapitalised": float,
    "OtherExpeses": float,
    "PL_Before_OtherInc_Int_Excpltem_Tax": float,
    "PL_Before_Int_ExcpItem_Tax": float,
    "InterestCharges": float,
    "PL_Before_ExcpItem_Tax": float,
    "ExceptionalItems": float,
    "Depreciation": float,
    "OperatingProfitBeforeProvisionsAndContingencies": float,
    "BankProvisionsMade": float,
    "PL_Before_Tax": float,
    "TaxCharges": float,
    "PL_After_TaxFromOrdineryActivities": float,
    "ExtraOrdinaryItem": float,
    "ReportedPAT": float,
    "MinorityInterest": float,
    "ShareOfPLOfAssociates": float,
    "NetPLAfterMIAssociates": float,
    "CostOfInvestInSubsidiary": float,
    "PriorYearAdj": float,
    "ReservesWrittenBack": float,
    "EquityCapital": float,
    "ReservesAndSurplus": float,
    "EqDividendRate": float,
    "AggregateOfNonPromotoNoOfShares": float,
    "AggregateOfNonPromotoHoldingPercent": float,
    "GovernmentShare": float,
    "CapitalAdequacyRatio": float,
    "CapitalAdequacyBasell": float,
    "GrossNPA": float,
    "NetNPA": float,
    "Per_OfGrossNPA": float,
    "Per_OfNetNPA": float,
    "ReturnOnAssets_Per": float,
    "BeforeBasicEPS": float,
    "asBeforeDilutedEPS": float,
    "AfterBasicEPS": float,
    "AfterDilutedEPS": float,
    "En_NumberOfShares": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "NonEn_NumberOfShares": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "ModifiedDate": pd.to_datetime,
},
"ConsolidatedNineMonthsResults": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Nine": int,
    "OperatingIncome": float,
    "OtherOperatingIncome": float,
    "TotalIncomeFromOperations": float,
    "IntOrDiscOnAdvOrBills": float,
    "IncomeOnInvestements": float,
    "IntOnBalancesWithRBI": float,
    "Others": float,
    "OtherRecurringIncome": float,
    "StockAdjustment": float,
    "RawMaterialConsumed": float,
    "PurchaseOfTradedGoods": float,
    "PowerAndFuel": float,
    "EmployeeExpenses": float,
    "Excise": float,
    "AdminAndSellingExpenses": float,
    "ResearchAndDevelopmentExpenses": float,
    "ExpensesCapitalised": float,
    "OtherExpeses": float,
    "PL_Before_OtherInc_Int_Excpltem_Tax": float,
    "PL_Before_Int_ExcpItem_Tax": float,
    "InterestCharges": float,
    "PL_Before_ExcpItem_Tax": float,
    "ExceptionalItems": float,
    "Depreciation": float,
    "OperatingProfitBeforeProvisionsAndContingencies": float,
    "BankProvisionsMade": float,
    "PL_Before_Tax": float,
    "TaxCharges": float,
    "PL_After_TaxFromOrdineryActivities": float,
    "ExtraOrdinaryItem": float,
    "ReportedPAT": float,
    "MinorityInterest": float,
    "ShareOfPLOfAssociates": float,
    "NetPLAfterMIAssociates": float,
    "CostOfInvestInSubsidiary": float,
    "PriorYearAdj": float,
    "ReservesWrittenBack": float,
    "EquityCapital": float,
    "ReservesAndSurplus": float,
    "EqDividendRate": float,
    "AggregateOfNonPromotoNoOfShares": float,
    "AggregateOfNonPromotoHoldingPercent": float,
    "GovernmentShare": float,
    "CapitalAdequacyRatio": float,
    "CapitalAdequacyBasell": float,
    "GrossNPA": float,
    "NetNPA": float,
    "Per_OfGrossNPA": float,
    "Per_OfNetNPA": float,
    "ReturnOnAssets_Per": float,
    "BeforeBasicEPS": float,
    "asBeforeDilutedEPS": float,
    "AfterBasicEPS": float,
    "AfterDilutedEPS": float,
    "En_NumberOfShares": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "NonEn_NumberOfShares": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "ModifiedDate": pd.to_datetime,
},
"ConsolidatedQuarterlyEPS": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "Sales": float,
    "Expenses": float,
    "EBIDTA": float,
    "Interest": float,
    "Depreciation": float,
    "Extraordinary": float,
    "OPM": float,
    "Tax": float,
    "PATRAW": float,
    "PAT": float,
    "Equity": float,
    "Reserves": float,
    "EPS": float,
    "NPM": float,
    "Ext_Flag": bool,
    "Q1 EPS Growth": float,
    "Q1 Sales Growth": float,
    "Q2 EPS": float,
    "Q2 EPS Growth": float,
    "Q2 Sales": float,
    "Q2 Sales Growth": float,
},
"ConsolidatedQuarterlyEPS_DUPLICATED": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "Sales": float,
    "Expenses": float,
    "EBIDTA": float,
    "Interest": float,
    "Depreciation": float,
    "Extraordinary": float,
    "OPM": float,
    "Tax": float,
    "PATRAW": float,
    "PAT": float,
    "Equity": float,
    "Reserves": float,
    "EPS": float,
    "NPM": float,
    "Ext_Flag": bool,
    "Q1 EPS Growth": float,
    "Q1 Sales Growth": float,
    "Q2 EPS": float,
    "Q2 EPS Growth": float,
    "Q2 Sales": float,
    "Q2 Sales Growth": float,
},
"ConsolidatedQuarterlyResults": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "OperatingIncome": float,
    "OtherOperatingIncome": float,
    "TotalIncomeFromOperations": float,
    "IntOrDiscOnAdvOrBills": float,
    "IncomeOnInvestements": float,
    "IntOnBalancesWithRBI": float,
    "Others": float,
    "OtherRecurringIncome": float,
    "StockAdjustment": float,
    "RawMaterialConsumed": float,
    "PurchaseOfTradedGoods": float,
    "PowerAndFuel": float,
    "EmployeeExpenses": float,
    "Excise": float,
    "AdminAndSellingExpenses": float,
    "ResearchAndDevelopmentExpenses": float,
    "ExpensesCapitalised": float,
    "OtherExpeses": float,
    "PL_Before_OtherInc_Int_Excpltem_Tax": float,
    "PL_Before_Int_ExcpItem_Tax": float,
    "InterestCharges": float,
    "PL_Before_ExcpItem_Tax": float,
    "ExceptionalItems": float,
    "Depreciation": float,
    "OperatingProfitBeforeProvisionsAndContingencies": float,
    "BankProvisionsMade": float,
    "PL_Before_Tax": float,
    "TaxCharges": float,
    "PL_After_TaxFromOrdineryActivities": float,
    "ExtraOrdinaryItem": float,
    "ReportedPAT": float,
    "MinorityInterest": float,
    "ShareOfPLOfAssociates": float,
    "NetPLAfterMIAssociates": float,
    "CostOfInvestInSubsidiary": float,
    "PriorYearAdj": float,
    "ReservesWrittenBack": float,
    "EquityCapital": float,
    "ReservesAndSurplus": float,
    "EqDividendRate": float,
    "AggregateOfNonPromotoNoOfShares": float,
    "AggregateOfNonPromotoHoldingPercent": float,
    "GovernmentShare": float,
    "CapitalAdequacyRatio": float,
    "CapitalAdequacyBasell": float,
    "GrossNPA": float,
    "NetNPA": float,
    "Per_OfGrossNPA": float,
    "Per_OfNetNPA": float,
    "ReturnOnAssets_Per": float,
    "BeforeBasicEPS": float,
    "asBeforeDilutedEPS": float,
    "AfterBasicEPS": float,
    "AfterDilutedEPS": float,
    "En_NumberOfShares": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "NonEn_NumberOfShares": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "ModifiedDate": pd.to_datetime,
},
"ConsolidatedQuarterlyResults_DUPLICATED": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "OperatingIncome": float,
    "OtherOperatingIncome": float,
    "TotalIncomeFromOperations": float,
    "IntOrDiscOnAdvOrBills": float,
    "IncomeOnInvestements": float,
    "IntOnBalancesWithRBI": float,
    "Others": float,
    "OtherRecurringIncome": float,
    "StockAdjustment": float,
    "RawMaterialConsumed": float,
    "PurchaseOfTradedGoods": float,
    "PowerAndFuel": float,
    "EmployeeExpenses": float,
    "Excise": float,
    "AdminAndSellingExpenses": float,
    "ResearchAndDevelopmentExpenses": float,
    "ExpensesCapitalised": float,
    "OtherExpeses": float,
    "PL_Before_OtherInc_Int_Excpltem_Tax": float,
    "PL_Before_Int_ExcpItem_Tax": float,
    "InterestCharges": float,
    "PL_Before_ExcpItem_Tax": float,
    "ExceptionalItems": float,
    "Depreciation": float,
    "OperatingProfitBeforeProvisionsAndContingencies": float,
    "BankProvisionsMade": float,
    "PL_Before_Tax": float,
    "TaxCharges": float,
    "PL_After_TaxFromOrdineryActivities": float,
    "ExtraOrdinaryItem": float,
    "ReportedPAT": float,
    "MinorityInterest": float,
    "ShareOfPLOfAssociates": float,
    "NetPLAfterMIAssociates": float,
    "CostOfInvestInSubsidiary": float,
    "PriorYearAdj": float,
    "ReservesWrittenBack": float,
    "EquityCapital": float,
    "ReservesAndSurplus": float,
    "EqDividendRate": float,
    "AggregateOfNonPromotoNoOfShares": float,
    "AggregateOfNonPromotoHoldingPercent": float,
    "GovernmentShare": float,
    "CapitalAdequacyRatio": float,
    "CapitalAdequacyBasell": float,
    "GrossNPA": float,
    "NetNPA": float,
    "Per_OfGrossNPA": float,
    "Per_OfNetNPA": float,
    "ReturnOnAssets_Per": float,
    "BeforeBasicEPS": float,
    "asBeforeDilutedEPS": float,
    "AfterBasicEPS": float,
    "AfterDilutedEPS": float,
    "En_NumberOfShares": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "NonEn_NumberOfShares": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "ModifiedDate": pd.to_datetime,
},
"ConsolidatedQuarterlyResults_OLD": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "OperatingIncome": float,
    "OtherOperatingIncome": float,
    "TotalIncomeFromOperations": float,
    "IntOrDiscOnAdvOrBills": float,
    "IncomeOnInvestements": float,
    "IntOnBalancesWithRBI": float,
    "Others": float,
    "OtherRecurringIncome": float,
    "StockAdjustment": float,
    "RawMaterialConsumed": float,
    "PurchaseOfTradedGoods": float,
    "PowerAndFuel": float,
    "EmployeeExpenses": float,
    "Excise": float,
    "AdminAndSellingExpenses": float,
    "ResearchAndDevelopmentExpenses": float,
    "ExpensesCapitalised": float,
    "OtherExpeses": float,
    "PL_Before_OtherInc_Int_Excpltem_Tax": float,
    "PL_Before_Int_ExcpItem_Tax": float,
    "InterestCharges": float,
    "PL_Before_ExcpItem_Tax": float,
    "ExceptionalItems": float,
    "Depreciation": float,
    "OperatingProfitBeforeProvisionsAndContingencies": float,
    "BankProvisionsMade": float,
    "PL_Before_Tax": float,
    "TaxCharges": float,
    "PL_After_TaxFromOrdineryActivities": float,
    "ExtraOrdinaryItem": float,
    "ReportedPAT": float,
    "MinorityInterest": float,
    "ShareOfPLOfAssociates": float,
    "NetPLAfterMIAssociates": float,
    "CostOfInvestInSubsidiary": float,
    "PriorYearAdj": float,
    "ReservesWrittenBack": float,
    "EquityCapital": float,
    "ReservesAndSurplus": float,
    "EqDividendRate": float,
    "AggregateOfNonPromotoNoOfShares": float,
    "AggregateOfNonPromotoHoldingPercent": float,
    "GovernmentShare": float,
    "CapitalAdequacyRatio": float,
    "CapitalAdequacyBasell": float,
    "GrossNPA": float,
    "NetNPA": float,
    "Per_OfGrossNPA": float,
    "Per_OfNetNPA": float,
    "ReturnOnAssets_Per": float,
    "BeforeBasicEPS": float,
    "asBeforeDilutedEPS": float,
    "AfterBasicEPS": float,
    "AfterDilutedEPS": float,
    "En_NumberOfShares": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "NonEn_NumberOfShares": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "ModifiedDate": pd.to_datetime,
},
"ConsolidatedTTM": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "Sales": float,
    "Expenses": float,
    "EBIDTA": float,
    "Interest": float,
    "Depreciation": float,
    "Extraordinary": float,
    "OPM": float,
    "Tax": float,
    "PAT": float,
    "Equity": float,
    "Reserves": float,
    "EPS": float,
    "NPM": float,
},
"ConsolidatedTTM_DUPLICATED": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "Sales": float,
    "Expenses": float,
    "EBIDTA": float,
    "Interest": float,
    "Depreciation": float,
    "Extraordinary": float,
    "OPM": float,
    "Tax": float,
    "PAT": float,
    "Equity": float,
    "Reserves": float,
    "EPS": float,
    "NPM": float,
},
"CurrentData": {
    "Companycode": float,
    "ResultType": str,
    "Yearending": pd.to_datetime,
    "Months": int,
    "Facevalue": float,
    "BonusEquity": float,
    "NumberOfEquityShares": float,
    "CurrentEquity": float,
    "CurrentReserves": float,
    "TotalDebt": float,
    "CashAndBankBalance": float,
    "Sales": float,
    "PrevYearSales": float,
    "OperatingProfit": float,
    "GrossProfit": float,
    "Depriciation": float,
    "Tax": float,
    "ReportedPAT": float,
    "PrevYearReportedPAT": float,
    "Dividend": float,
    "DividendPerShare": float,
    "EPS": float,
    "BookValue": float,
    "Price": float,
    "PriceDate": pd.to_datetime,
    "IndustryPE": float,
    "BetaValue": float,
    "PledgedShares": float,
    "TrailingYear": pd.to_datetime,
    "LastFourQuarterersSales": float,
    "LastFourQuartersTotalExpenditure": float,
    "LastFourQuartersInterestCharges": float,
    "LastFourQuartersGrossProfit": float,
    "LastFourQuartersDepreciation": float,
    "LastFourQuartersBankProvisionsMade": float,
    "LastFourQuartersTaxCharges": float,
    "LastFourQuartersExtraOrdinaryItem": float,
    "LastFourQuartersReportedPAT": float,
    "LastFourQuartersPriorYearAdj": float,
    "TrailingEPS": float,
    "TrailingCEPS": float,
    "ModifiedDate": pd.to_datetime,
},
"Dividend": {
    "CompanyCode": float,
    "DateOfAnnouncement": pd.to_datetime,
    "InterimOrFinal": str,
    "InstrumentType": int,
    "InstrumentTypeDescription": str,
    "Percentage": float,
    "Value": float,
    "RecordDate": pd.to_datetime,
    "BookClosureStartDate": pd.to_datetime,
    "BookClosureEndDate": pd.to_datetime,
    "XDDate": pd.to_datetime,
    "Remarks": str,
    "DelFlag": bool,
    "ModifiedDate": pd.to_datetime,
},
"ErrorLog": {
    "TIMESTAMP": pd.to_datetime,
    "STATUS": str,
    "REPORT": str,
},
"FinanceBankingVI": {
    "Companycode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Type": str,
    "FaceValue": float,
    "InterestDiscountOnAdvancesBills": float,
    "IncomeFromInvestments": float,
    "InterestOnBalanceWithRBIAndOtherInterBankFunds": float,
    "Others": float,
    "TotalInterestEarned": float,
    "OtherIncome": float,
    "TotalIncome": float,
    "InterestExpended": float,
    "PaymentsToAndProvisionsForEmployees": float,
    "Depreciation": float,
    "DepreciationOnLeasedAssets": float,
    "OperatingExpensesExcludesEmployeeCostAndDepreciation": float,
    "TotalOperatingExpenses": float,
    "ProvisionTowardsIncomeTax": float,
    "ProvisionTowardsDeferredTax": float,
    "ProvisionTowardsOtherTaxes": float,
    "OtherProvisionsAndContingencies": float,
    "TotalProvisionsAndContingencies": float,
    "TotalExpenditure": float,
    "NetProfitLossForTheYear": float,
    "PriorPeriodItems": float,
    "ExtraordinaryItems": float,
    "NetProfitLossForTheYearAfterEI": float,
    "ProfitLossBroughtForward": float,
    "TransferredOnAmalgamation": float,
    "TotalProfitLoss": float,
    "TransferToFromStatutoryReserve": float,
    "TransferToFromReserveFund": float,
    "TransferToFromSpecialReserve": float,
    "TransferToFromCapitalReserve": float,
    "TransferToFromGeneralReserve": float,
    "TransferToFromInvestmentReserve": float,
    "TransferToFromRevenueAndOtherReserves": float,
    "DividendForThePreviousYear": float,
    "EquityShareDividend": float,
    "PreferenceShareDividend": float,
    "TaxOnDividend": float,
    "BalanceCarriedOverToBalanceSheet": float,
    "TotalAppropriations": float,
    "EquityShares": float,
    "EquityCapital": float,
    "PreferenceCapital": float,
    "TotalShareCapital": float,
    "RevaluationReserves": float,
    "ReservesAndSurplus": float,
    "TotalReservesAndSurplus": float,
    "MoneyAgainstShareWarrants": float,
    "EmployeesStockOptions": float,
    "TotalShareHoldersFunds": float,
    "EquityShareApplicationMoney": float,
    "PrefShareApplicationMoney": float,
    "ShareCapitalSuspense": float,
    "Deposits": float,
    "Borrowings": float,
    "OtherLiabilitiesAndProvisions": float,
    "TotalCapitalAndLiabilities": float,
    "CashAndBalancesWithReserveBankOfIndia": float,
    "BalancesWithBanksMoneyAtCallAndShortNotice": float,
    "Investments": float,
    "Advances": float,
    "FixedAssets": float,
    "OtherAssets": float,
    "TotalAssets": float,
    "NetProfitLossBeforeExtraordinaryItemsAndTax": float,
    "NetCashFlowFromOperatingActivities": float,
    "NetCashUsedInInvestingActivities": float,
    "NetCashUsedFromFinancingActivities": float,
    "ForeignExchangeGainsLosses": float,
    "NetIncDecInCashAndCashEquivalents": float,
    "CashAndCashEquivalentBeginOfYear": float,
    "CashAndCashEquivalentEndOfYear": float,
    "BasicEPS": float,
    "DilutedEPS": float,
    "NumberOfBranches": float,
    "NumberOfEmployees": float,
    "CapitalAdequacyRatios": float,
    "KeyPerformanceTier1": float,
    "KeyPerformanceTier2": float,
    "GrossNPARs": float,
    "GrossNPAPercentage": float,
    "NetNPARs": float,
    "NetNPAPercentage": float,
    "NetNPAToAdvancesPercentage": float,
    "BillsForCollection": float,
    "ContingentLiabilities": float,
    "EquityDividendRate": float,
    "BonusEquityShareCapital": float,
    "ModifiedDate": pd.to_datetime,
},
"FinanceConsolidatedBankingVI": {
    "Companycode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Type": str,
    "FaceValue": float,
    "InterestDiscountOnAdvancesBills": float,
    "IncomeFromInvestments": float,
    "InterestOnBalanceWithRBIAndOtherInterBankFunds": float,
    "Others": float,
    "TotalInterestEarned": float,
    "OtherIncome": float,
    "TotalIncome": float,
    "InterestExpended": float,
    "PaymentsToAndProvisionsForEmployees": float,
    "Depreciation": float,
    "DepreciationOnLeasedAssets": float,
    "OperatingExpensesExcludesEmployeeCostAndDepreciation": float,
    "TotalOperatingExpenses": float,
    "ProvisionTowardsIncomeTax": float,
    "ProvisionTowardsDeferredTax": float,
    "ProvisionTowardsOtherTaxes": float,
    "OtherProvisionsAndContingencies": float,
    "TotalProvisionsAndContingencies": float,
    "TotalExpenditure": float,
    "NetProfitLossForTheYear": float,
    "PriorPeriodItems": float,
    "ExtraordinaryItems": float,
    "NetProfitLossForTheYearAfterEI": float,
    "IncomeMinorityInterest": float,
    "ShareOfProfitLossFromAssociate": float,
    "ProfitLossAfterMIAndAssociates": float,
    "ProfitLossBroughtForward": float,
    "TransferredOnAmalgamation": float,
    "TotalProfitLoss": float,
    "TransferToFromStatutoryReserve": float,
    "TransferToFromReserveFund": float,
    "TransferToFromSpecialReserve": float,
    "TransferToFromCapitalReserve": float,
    "TransferToFromGeneralReserve": float,
    "TransferToFromInvestmentReserve": float,
    "TransferToFromRevenueAndOtherReserves": float,
    "DividendForThePreviousYear": float,
    "EquityShareDividend": float,
    "PreferenceShareDividend": float,
    "TaxOnDividend": float,
    "BalanceCarriedOverToBalanceSheet": float,
    "TotalAppropriations": float,
    "EquityShares": float,
    "EquityCapital": float,
    "PreferenceCapital": float,
    "TotalShareCapital": float,
    "RevaluationReserves": float,
    "ReservesAndSurplus": float,
    "TotalReservesAndSurplus": float,
    "MoneyAgainstShareWarrants": float,
    "EmployeesStockOptions": float,
    "TotalShareHoldersFunds": float,
    "EquityShareApplicationMoney": float,
    "PrefShareApplicationMoney": float,
    "ShareCapitalSuspense": float,
    "LiabilityMinorityInterest": float,
    "PolicyHoldersFunds": float,
    "LiabilityGroupShareInJointVentures": float,
    "Deposits": float,
    "Borrowings": float,
    "OtherLiabilitiesAndProvisions": float,
    "TotalCapitalAndLiabilities": float,
    "CashAndBalancesWithReserveBankOfIndia": float,
    "BalancesWithBanksMoneyAtCallAndShortNotice": float,
    "Investments": float,
    "Advances": float,
    "FixedAssets": float,
    "OtherAssets": float,
    "AssetMinorityInterest": float,
    "AssetGroupShareInJointVentures": float,
    "TotalAssets": float,
    "NetProfitLossBeforeExtraordinaryItemsAndTax": float,
    "NetCashFlowFromOperatingActivities": float,
    "NetCashUsedInInvestingActivities": float,
    "NetCashUsedFromFinancingActivities": float,
    "ForeignExchangeGainsLosses": float,
    "NetIncDecInCashAndCashEquivalents": float,
    "CashAndCashEquivalentBeginOfYear": float,
    "CashAndCashEquivalentEndOfYear": float,
    "BasicEPS": float,
    "DilutedEPS": float,
    "NumberOfBranches": float,
    "NumberOfEmployees": float,
    "CapitalAdequacyRatios": float,
    "KeyPerformanceTier1": float,
    "KeyPerformanceTier2": float,
    "GrossNPARs": float,
    "GrossNPAPercentage": float,
    "NetNPARs": float,
    "NetNPAPercentage": float,
    "NetNPAToAdvancesPercentage": float,
    "BillsForCollection": float,
    "ContingentLiabilities": float,
    "BonusEquityShareCapital": float,
    "ModifiedDate": pd.to_datetime,
},
"FinanceConsolidatedNonBankingVI": {
    "Companycode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Type": str,
    "FaceValue": float,
    "RevenueFromOperationsGross": float,
    "LessExciseSeviceTaxOtherLevies": float,
    "RevenueFromOperationsNet": float,
    "OtherOperatingRevenues": float,
    "TotalOperatingRevenues": float,
    "OtherIncome": float,
    "GroupShareInJointVenturesInIncome": float,
    "TotalRevenue": float,
    "CostOfMaterialsConsumed": float,
    "PurchaseOfStockInTrade": float,
    "PurchaseOfCrudeOilAndOthers": float,
    "CostOfPowerPurchased": float,
    "CostOfFuel": float,
    "AircraftFuelExpenses": float,
    "AircraftLeaseRentals": float,
    "OperatingAndDirectExpenses": float,
    "ChangesIninventoriesOfFGWIPAndStockInTrade": float,
    "EmployeeBenefitExpenses": float,
    "FinanceCosts": float,
    "ProvisionsAndContingencies": float,
    "DepreciationAndAmortisationExpenses": float,
    "MiscExpensesWOff": float,
    "OtherExpenses": float,
    "LessInterUnitSegmentDivisionTransfer": float,
    "LessTransferToFromInvestmentFixedAssetsOthers": float,
    "LessAmountsTransferToCapitalAccounts": float,
    "LessShareOfLossFromPartnershipFirm": float,
    "GroupShareInJointVenturesInExpenditure": float,
    "TotalExpenses": float,
    "ProfitLossBeforeExceptionalExtraOrdinaryItemsAndTax": float,
    "ExceptionalItems": float,
    "ProfitLossBeforeTax": float,
    "CurrentTax": float,
    "LessMATCredit": float,
    "DeferredTax": float,
    "OtherDirectTaxes": float,
    "TaxForEarlierYears": float,
    "TotalTaxExpensesContinuedOperations": float,
    "ProfitLossAfterTaxAndBeforeExtraOrdinaryItems": float,
    "PriorPeriodItems": float,
    "ExtraordinaryItems": float,
    "ProfitLossFromContinuingOperations": float,
    "ProfitLossFromDiscontinuingOperations": float,
    "TotalTaxExpensesDiscontinuingOperations": float,
    "NetProfitLossFromDiscontinuingOperations": float,
    "ProfitLossForThePeriod": float,
    "IncomeMinorityInterest": float,
    "ShareOfProfitLossOfAssociates": float,
    "ProfitLossAfterMIAndAssociates": float,
    "EquityShares": float,
    "EquityCapital": float,
    "PreferenceCapital": float,
    "TotalShareCapital": float,
    "RevaluationReserves": float,
    "ReservesAndSurplus": float,
    "TotalReservesAndSurplus": float,
    "MoneyAgainstShareWarrants": float,
    "EmployeesStockOptions": float,
    "TotalShareHoldersFunds": float,
    "PrefSharesIssuedBySubsidiaryCos": float,
    "EquityShareApplicationMoney": float,
    "PreferenceShareApplicationMoney": float,
    "ShareCapitalSuspense": float,
    "HybridDebtOtherSecurities": float,
    "StatutoryConsumerReserves": float,
    "SpecialApprnTowardsProjectCost": float,
    "ServiceLineContributionFromConsumers": float,
    "GovernmentOtherGrants": float,
    "LiabilityMinorityInterest": float,
    "PolicyHoldersFunds": float,
    "LiabilityGroupShareInJointVentures": float,
    "LongTermBorrowings": float,
    "DeferredTaxLiabilities": float,
    "OtherLongTermLiabilities": float,
    "LongTermProvisions": float,
    "TotalNonCurrentLiabilities": float,
    "ForeignCurrencyMonetaryItemTranslationDifferenceLiabilityAccoun": float,
    "ShortTermBorrowings": float,
    "TradePayables": float,
    "OtherCurrentLiabilities": float,
    "ShortTermProvisions": float,
    "TotalCurrentLiabilities": float,
    "TotalCapitalAndLiabilities": float,
    "TangibleAssets": float,
    "IntangibleAssets": float,
    "CapitalWorkInProgress": float,
    "IntangibleAssetsUnderDevelopment": float,
    "OtherAssets": float,
    "NonCurrentInvestmentsUnquotedBookValue": float,
    "ConstructionStores": float,
    "MiningDevelopmentExpenditure": float,
    "AssetsHeldForSale": float,
    "TotalFixedAssets": float,
    "GoodwillOnConsolidation": float,
    "NonCurrentInvestments": float,
    "DeferredTaxAssets": float,
    "LongTermLoansAndAdvances": float,
    "OtherNonCurrentAssets": float,
    "TotalNonCurrentAssets": float,
    "AssetMinorityInterest": float,
    "AssetGroupShareInJointVentures": float,
    "ForeignCurrencyMonetaryItemTranslationDiffAcct": float,
    "CurrentInvestments": float,
    "Inventories": float,
    "TradeReceivables": float,
    "CashAndCashEquivalents": float,
    "ShortTermLoansAndAdvances": float,
    "OtherCurrentAssets": float,
    "TotalCurrentAssets": float,
    "TotalAssets": float,
    "NetProfitLossBeforeExtraordinaryItemsAndTax": float,
    "NetCashFlowFromOperatingActivities": float,
    "NetCashUsedInInvestingActivities": float,
    "NetCashUsedFromFinancingActivities": float,
    "ForeignExchangeGainsLosses": float,
    "AdjustmentsOnAmalgamationMergerDemergerOthers": float,
    "NetIncDecInCashAndCashEquivalents": float,
    "CashAndCashEquivalentBeginOfYear": float,
    "CashAndCashEquivalentEndOfYear": float,
    "BasicEPS": float,
    "DilutedEPS": float,
    "ContingentLiabilities": float,
    "CIFValueOfRawMaterials": float,
    "CIFValueOfStoresSparesAndLooseTools": float,
    "CIFValueOfOtherGoods": float,
    "CIFValueOfCapitalGoods": float,
    "ExpenditureInForeignCurrency": float,
    "DividendRemittanceInForeignCurrency": float,
    "ForeignExchangeFOB": float,
    "ForeignExchangeOtherEarnings": float,
    "ImportedRawMaterials": float,
    "IndigenousRawMaterials": float,
    "ImportedStoresAndSpares": float,
    "IndigenousStoresAndSpares": float,
    "EquityShareDividend": float,
    "PreferenceShareDividend": float,
    "TaxOnDividend": float,
    "BonusEquityShareCapital": float,
    "NonCurrentInvestmentsQuotedMarketValue": float,
    "CurrentInvestmentsQuotedMarketValue": float,
    "CurrentInvestmentsUnquotedBookValue": float,
    "ModifiedDate": pd.to_datetime,
},
"FinanceNonBankingVI": {
    "Companycode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Type": str,
    "FaceValue": float,
    "RevenueFromOperationsGross": float,
    "LessExciseSeviceTaxOtherLevies": float,
    "RevenueFromOperationsNet": float,
    "OtherOperatingRevenues": float,
    "TotalOperatingRevenues": float,
    "OtherIncome": float,
    "TotalRevenue": float,
    "CostOfMaterialsConsumed": float,
    "PurchaseOfStockInTrade": float,
    "PurchaseOfCrudeOilAndOthers": float,
    "CostOfPowerPurchased": float,
    "CostOfFuel": float,
    "AircraftFuelExpenses": float,
    "AircraftLeaseRentals": float,
    "OperatingAndDirectExpenses": float,
    "ChangesIninventoriesOfFGWIPAndStockInTrade": float,
    "EmployeeBenefitExpenses": float,
    "FinanceCosts": float,
    "ProvisionsAndContingencies": float,
    "DepreciationAndAmortisationExpenses": float,
    "MiscExpensesWOff": float,
    "OtherExpenses": float,
    "LessInterUnitSegmentDivisionTransfer": float,
    "LessTransferToFromInvestmentFixedAssetsOthers": float,
    "LessAmountsTransferToCapitalAccounts": float,
    "LessShareOfLossFromPartnershipFirm": float,
    "TotalExpenses": float,
    "ProfitLossBeforeExceptionalExtraOrdinaryItemsAndTax": float,
    "ExceptionalItems": float,
    "ProfitLossBeforeTax": float,
    "CurrentTax": float,
    "LessMATCredit": float,
    "DeferredTax": float,
    "OtherDirectTaxes": float,
    "TaxForEarlierYears": float,
    "TotalTaxExpensesContinuedOperations": float,
    "ProfitLossAfterTaxAndBeforeExtraOrdinaryItems": float,
    "PriorPeriodItems": float,
    "ExtraordinaryItems": float,
    "ProfitLossFromContinuingOperations": float,
    "ProfitLossFromDiscontinuingOperations": float,
    "TotalTaxExpensesDiscontinuingOperations": float,
    "NetProfitLossFromDiscontinuingOperations": float,
    "ProfitLossForThePeriod": float,
    "EquityShares": float,
    "EquityCapital": float,
    "PreferenceCapital": float,
    "TotalShareCapital": float,
    "RevaluationReserves": float,
    "ReservesAndSurplus": float,
    "TotalReservesAndSurplus": float,
    "MoneyAgainstShareWarrants": float,
    "EmployeesStockOptions": float,
    "TotalShareHoldersFunds": float,
    "EquityShareApplicationMoney": float,
    "PreferenceShareApplicationMoney": float,
    "ShareCapitalSuspense": float,
    "HybridDebtOtherSecurities": float,
    "StatutoryConsumerReserves": float,
    "SpecialApprnTowardsProjectCost": float,
    "ServiceLineContributionFromConsumers": float,
    "GovernmentOtherGrants": float,
    "LongTermBorrowings": float,
    "DeferredTaxLiabilities": float,
    "OtherLongTermLiabilities": float,
    "LongTermProvisions": float,
    "TotalNonCurrentLiabilities": float,
    "ForeignCurrencyMonetaryItemTranslationDifferenceLiabilityAccoun": float,
    "ShortTermBorrowings": float,
    "TradePayables": float,
    "OtherCurrentLiabilities": float,
    "ShortTermProvisions": float,
    "TotalCurrentLiabilities": float,
    "TotalCapitalAndLiabilities": float,
    "TangibleAssets": float,
    "IntangibleAssets": float,
    "CapitalWorkInProgress": float,
    "IntangibleAssetsUnderDevelopment": float,
    "OtherAssets": float,
    "ConstructionStores": float,
    "MiningDevelopmentExpenditure": float,
    "AssetsHeldForSale": float,
    "TotalFixedAssets": float,
    "NonCurrentInvestments": float,
    "DeferredTaxAssets": float,
    "LongTermLoansAndAdvances": float,
    "OtherNonCurrentAssets": float,
    "TotalNonCurrentAssets": float,
    "ForeignCurrencyMonetaryItemTranslationDiffAcct": float,
    "CurrentInvestments": float,
    "Inventories": float,
    "TradeReceivables": float,
    "CashAndCashEquivalents": float,
    "ShortTermLoansAndAdvances": float,
    "OtherCurrentAssets": float,
    "TotalCurrentAssets": float,
    "TotalAssets": float,
    "NetProfitLossBeforeExtraordinaryItemsAndTax": float,
    "NetCashFlowFromOperatingActivities": float,
    "NetCashUsedInInvestingActivities": float,
    "NetCashUsedFromFinancingActivities": float,
    "ForeignExchangeGainsLosses": float,
    "AdjustmentsOnAmalgamationMergerDemergerOthers": float,
    "NetIncDecInCashAndCashEquivalents": float,
    "CashAndCashEquivalentBeginOfYear": float,
    "CashAndCashEquivalentEndOfYear": float,
    "BasicEPS": float,
    "DilutedEPS": float,
    "ContingentLiabilities": float,
    "CIFValueOfRawMaterials": float,
    "CIFValueOfStoresSparesAndLooseTools": float,
    "CIFValueOfOtherGoods": float,
    "CIFValueOfCapitalGoods": float,
    "ExpenditureInForeignCurrency": float,
    "DividendRemittanceInForeignCurrency": float,
    "ForeignExchangeFOB": float,
    "ForeignExchangeOtherEarnings": float,
    "ImportedRawMaterials": float,
    "IndigenousRawMaterials": float,
    "ImportedStoresAndSpares": float,
    "IndigenousStoresAndSpares": float,
    "EquityShareDividend": float,
    "PreferenceShareDividend": float,
    "TaxOnDividend": float,
    "EquityDividendRate": float,
    "BonusEquityShareCapital": float,
    "NonCurrentInvestmentsQuotedMarketValue": float,
    "NonCurrentInvestmentsUnquotedBookValue": float,
    "CurrentInvestmentsQuotedMarketValue": float,
    "CurrentInvestmentsUnquotedBookValue": float,
    "ModifiedDate": pd.to_datetime,
},
"FundMaster": {
    "CompanyCode": float,
    "CompanyName": str,
    "SponsorName": str,
    "TrusteeCompanyName": str,
    "MFSetUpDate": pd.to_datetime,
    "AMCCode": float,
    "AMCName": str,
    "AMCIncDate": pd.to_datetime,
    "CEOName": str,
    "CIOName": str,
    "FundManagerName": str,
    "ComplianceOfficerName": str,
    "InvServiceOfficerName": str,
    "AuditorsName": str,
    "CustodianName": str,
    "RegistrarName": str,
    "TypeOfMutualFund": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
},
"HalfYearlyResults": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Half": int,
    "OperatingIncome": float,
    "OtherOperatingIncome": float,
    "TotalIncomeFromOperations": float,
    "IntOrDiscOnAdvOrBills": float,
    "IncomeOnInvestements": float,
    "IntOnBalancesWithRBI": float,
    "Others": float,
    "OtherRecurringIncome": float,
    "StockAdjustment": float,
    "RawMaterialConsumed": float,
    "PurchaseOfTradedGoods": float,
    "PowerAndFuel": float,
    "EmployeeExpenses": float,
    "Excise": float,
    "AdminAndSellingExpenses": float,
    "ResearchAndDevelopmentExpenses": float,
    "ExpensesCapitalised": float,
    "OtherExpeses": float,
    "PL_Before_OtherInc_Int_Excpltem_Tax": float,
    "PL_Before_Int_ExcpItem_Tax": float,
    "InterestCharges": float,
    "PL_Before_ExcpItem_Tax": float,
    "ExceptionalItems": float,
    "Depreciation": float,
    "OperatingProfitBeforeProvisionsAndContingencies": float,
    "BankProvisionsMade": float,
    "PL_Before_Tax": float,
    "TaxCharges": float,
    "PL_After_TaxFromOrdineryActivities": float,
    "ExtraOrdinaryItem": float,
    "ReportedPAT": float,
    "PrioryearAdj": float,
    "ReservesWrittenBack": float,
    "EquityCapital": float,
    "ReservesAndSurplus": float,
    "EqDividendRate": float,
    "AggregateOfNonPromotoNoOfShares": float,
    "AggregateOfNonPromotoHoldingPercent": float,
    "GovernmentShare": float,
    "CapitalAdequacyRatio": float,
    "CapitalAdequacyBasell": float,
    "GrossNPA": float,
    "NetNPA": float,
    "Percentage_OfGrossNPA": float,
    "Percentage_OfNetNPA": float,
    "ReturnOnAssets_Per": float,
    "BeforeBasicEPS": float,
    "BeforeDilutedEPS": float,
    "AfterBasicEPS": float,
    "AfterDilutedEPS": float,
    "En_NumberOfShares": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "NonEn_NumberOfShares": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "ModifiedDate": pd.to_datetime,
},
"IndexHistory": {
    "TICKER": str,
    "DATE": pd.to_datetime,
    "OPEN": float,
    "HIGH": float,
    "LOW": float,
    "CLOSE": float,
    "VOL": float,
},
"IndexHistory_DUPLICATED": {
    "TICKER": str,
    "DATE": pd.to_datetime,
    "OPEN": float,
    "HIGH": float,
    "LOW": float,
    "CLOSE": float,
    "VOL": float,
},
"IndexOHLC": {
    "IndexName": str,
    "NSECode": str,
    "Date": pd.to_datetime,
    "Open": float,
    "High": float,
    "Low": float,
    "Close": float,
    "Points Change": float,
    "Change": float,
    "Volume": float,
    "Turnover": float,
    "PE": float,
    "PB": float,
    "Div Yield": float,
},
"IndexOHLC_DUPLICATED": {
    "IndexName": str,
    "NSECode": str,
    "Date": pd.to_datetime,
    "Open": float,
    "High": float,
    "Low": float,
    "Close": float,
    "Points Change": float,
    "Change": float,
    "Volume": float,
    "Turnover": float,
    "PE": float,
    "PB": float,
    "Div Yield": float,
},
"IndividualHolding": {
    "CompanyCode": float,
    "ShareHoldingDate": pd.to_datetime,
    "InstrumentDescription": str,
    "Name": str,
    "NoOfShares": float,
    "PromoterFlag": bool,
    "ModifiedDate": pd.to_datetime,
},
"IndustryDivisor": {
    "IndexName": str,
    "IndexValue": float,
    "Divisor": float,
    "Date": pd.to_datetime,
    "OS": float,
},
"SubIndustryDivisor": {
    "IndexName": str,
    "IndexValue": float,
    "Divisor": float,
    "Date": pd.to_datetime,
    "OS": float,
},
"IndustryIndexList": {
    "GenDate": pd.to_datetime,
    "IndustryIndexName": str,
    "Industry": str,
    "Open": float,
    "High": float,
    "Low": float,
    "Close": float,
    "Volume": float,
    "PE": float,
    "EPS": float,
    "CompanyCount": int,
    "OS": float,
    "Earnings Growth": float,
},
"SubIndustryIndexList": {
    "GenDate": pd.to_datetime,
    "SubIndustryIndexName": str,
    "SubIndustry": str,
    "Open": float,
    "High": float,
    "Low": float,
    "Close": float,
    "Volume": float,
    "PE": float,
    "EPS": float,
    "CompanyCount": int,
    "OS": float,
    "Earnings Growth": float,
},
"IndustryList": {
    "CompanyCode": float,
    "CompanyName": str,
    "NSECode": str,
    "BSECode": int,
    "ISIN": str,
    "Industry": str,
    "Sector": str,
    "SubSector": str,
    "Open": float,
    "High": float,
    "Low": float,
    "Close": float,
    "Volume": float,
    "PrevClose": float,
    "Change": float,
    "OS": float,
    "FaceValue": float,
    "FreeFloat": float,
    "GenDate": pd.to_datetime,
    "FF_Open": float,
    "FF_High": float,
    "FF_Low": float,
    "FF_Close": float,
    "IndustryIndexName": str,
    "SectorIndexName": str,
    "SubSectorIndexName": str,
    "PAT": float,
    "Equity": float,
    "OS_Close": float,
    "prev_pat": float,
    "prev_equity": float,
},
"IndustryList_DUPLICATED": {
    "CompanyCode": float,
    "CompanyName": str,
    "NSECode": str,
    "BSECode": int,
    "ISIN": str,
    "Industry": str,
    "Sector": str,
    "SubSector": str,
    "Open": float,
    "High": float,
    "Low": float,
    "Close": float,
    "Volume": float,
    "PrevClose": float,
    "Change": float,
    "OS": float,
    "FaceValue": float,
    "FreeFloat": float,
    "GenDate": pd.to_datetime,
    "FF_Open": float,
    "FF_High": float,
    "FF_Low": float,
    "FF_Close": float,
    "IndustryIndexName": str,
    "SectorIndexName": str,
    "SubSectorIndexName": str,
    "PAT": float,
    "Equity": float,
    "OS_Close": float,
    "prev_pat": float,
    "prev_equity": float,
},
"IndustryMapping": {
    "IndustryCode": int,
    "IndustryName": str,
    "BroadIndustryCode": int,
    "BroadIndustryName": str,
    "Industry": str,
    "Code": str,
    "SubSector": str,
    "SubSectorCode": str,
    "Sector": str,
    "SectorCode": str,
    "SectorIndexName": str,
    "SubSectorIndexName": str,
    "IndustryIndexName": str,
},
"IndustryMaster": {
    "IndustryCode": float,
    "IndustryName": str,
    "BroadIndustryCode": float,
    "BroadIndustryName": str,
    "MajorSectorCode": float,
    "MajorSectorName": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
},
"KeyExecutives": {
    "Companycode": float,
    "TitleDescription": str,
    "ExecutiveName": str,
    "DesignationDescription": str,
    "ModifiedDate": pd.to_datetime,
    "RowId": int,
},
"Locations": {
    "CompanyCode": float,
    "AddressType": str,
    "Address": str,
    "Address2": str,
    "Address3": str,
    "Address4": str,
    "CityName": str,
    "StateName": str,
    "CountryName": str,
    "Pincode": str,
    "TelephoneNumbers": str,
    "Phone2": str,
    "Phone3": str,
    "Phone4": str,
    "Telex": str,
    "Telex1": str,
    "Fax": str,
    "Fax1": str,
    "Grams": str,
    "EMailID": str,
    "WebSite": str,
    "ModifiedDate": pd.to_datetime,
},
"MFInvestments": {
    "MFDate": pd.to_datetime,
    "InstrumentType": str,
    "Purchase": float,
    "Sales": float,
    "NetInvestment": float,
    "ModifiedDate": pd.to_datetime,
},
"MFMergeList": {
    "SchemeCode": float,
    "SchemeName": str,
    "SchemePlanCode": int,
    "SchemeCategoryDescription": str,
    "MainCategory": str,
    "SchemeTypeDescription": str,
    "AUM": float,
    "HoldingDate": pd.to_datetime,
    "InvestedCompanyCode": float,
    "InvestedCompanyName": str,
    "Industry": str,
    "Quantity": float,
    "Percentage": float,
    "ISINCode": str,
    "Market Cap Class": str,
    "GenDate": pd.to_datetime,
    "Market Rank": float,
},
"ManagementTeam": {
    "Companycode": float,
    "TitleDescription": str,
    "PersonName": str,
    "DesignationDescription": str,
    "ModifiedDate": pd.to_datetime,
    "RowId": int,
},
"NSE": {
    "SYMBOL": str,
    "SERIES": str,
    "OPEN": float,
    "HIGH": float,
    "LOW": float,
    "CLOSE": float,
    "LAST": float,
    "PREVCLOSE": float,
    "TOTTRDQTY": float,
    "TOTTRDVAL": float,
    "TIMESTAMP": pd.to_datetime,
    "TOTALTRADES": float,
    "ISIN": str,
},
"News": {
    "CompanyCode": float,
    "NewsDate": pd.to_datetime,
    "Headlines": str,
    "SourceName": str,
    "NewsTypeDescription": str,
    "NewsClassfication": str,
    "Synopsis": str,
    "ModifiedDate": pd.to_datetime,
},
"NinemonthsResults": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Nine": int,
    "OperatingIncome": float,
    "OtherOperatingIncome": float,
    "TotalIncomeFromOperations": float,
    "IntOrDiscOnAdvOrBills": float,
    "IncomeOnInvestements": float,
    "IntOnBalancesWithRBI": float,
    "Others": float,
    "OtherRecurringIncome": float,
    "StockAdjustment": float,
    "RawMaterialConsumed": float,
    "PurchaseOfTradedGoods": float,
    "PowerAndFuel": float,
    "EmployeeExpenses": float,
    "Excise": float,
    "AdminAndSellingExpenses": float,
    "ResearchAndDevelopmentExpenses": float,
    "ExpensesCapitalised": float,
    "OtherExpeses": float,
    "PL_Before_OtherInc_Int_Excpltem_Tax": float,
    "PL_Before_Int_ExcpItem_Tax": float,
    "InterestCharges": float,
    "PL_Before_ExcpItem_Tax": float,
    "ExceptionalItems": float,
    "Depreciation": float,
    "OperatingProfitBeforeProvisionsAndContingencies": float,
    "BankProvisionsMade": float,
    "PL_Before_Tax": float,
    "TaxCharges": float,
    "PL_After_TaxFromOrdineryActivities": float,
    "ExtraOrdinaryItem": float,
    "ReportedPAT": float,
    "PrioryearAdj": float,
    "ReservesWrittenBack": float,
    "EquityCapital": float,
    "ReservesAndSurplus": float,
    "EqDividendRate": float,
    "AggregateOfNonPromotoNoOfShares": float,
    "AggregateOfNonPromotoHoldingPercent": float,
    "GovernmentShare": float,
    "CapitalAdequacyRatio": float,
    "CapitalAdequacyBasell": float,
    "GrossNPA": float,
    "NetNPA": float,
    "Percentage_OfGrossNPA": float,
    "Percentage_OfNetNPA": float,
    "ReturnOnAssets_Per": float,
    "BeforeBasicEPS": float,
    "BeforeDilutedEPS": float,
    "AfterBasicEPS": float,
    "AfterDilutedEPS": float,
    "En_NumberOfShares": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "NonEn_NumberOfShares": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "ModifiedDate": pd.to_datetime,
},
"OHLC": {
    "Company": str,
    "NSECode": str,
    "BSECode": int,
    "ISIN": str,
    "Open": float,
    "High": float,
    "Low": float,
    "Close": float,
    "Date": pd.to_datetime,
    "Value": float,
    "Volume": float,
    "CompanyCode": float,
},
"OHLC_DUPLICATED": {
    "Company": str,
    "NSECode": str,
    "BSECode": int,
    "ISIN": str,
    "Open": float,
    "High": float,
    "Low": float,
    "Close": float,
    "Date": pd.to_datetime,
    "Value": float,
    "Volume": int,
    "CompanyCode": float,
},
"PE": {
    "CompanyCode": float,
    "NSECode": str,
    "BSECode": int,
    "ISINCode": str,
    "Market Cap Value": float,
    "Market Cap Rank": int,
    "Market Cap Class": str,
    "PE": float,
    "PE High": float,
    "PE High Date": pd.to_datetime,
    "PE Low": float,
    "PE Low Date": pd.to_datetime,
    "GenDate": pd.to_datetime,
},
"PE_DUPLICATED": {
    "CompanyCode": float,
    "NSECode": str,
    "BSECode": int,
    "ISINCode": str,
    "Market Cap Value": float,
    "Market Cap Rank": int,
    "Market Cap Class": str,
    "PE": float,
    "PE High": float,
    "PE High Date": pd.to_datetime,
    "PE Low": float,
    "PE Low Date": pd.to_datetime,
    "GenDate": pd.to_datetime,
},
"PledgeShares": {
    "CompanyCode": float,
    "SHPDate": pd.to_datetime,
    "PledgeShares_Nos": int,
    "PledgeShares_Pct": float,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
},
"Products": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "NoOfMonths": int,
    "ProductCode": float,
    "ProductName": str,
    "LicensedCapacity": float,
    "LICUOM": str,
    "InstalledCapacity": float,
    "InstalledUOM": str,
    "ProductionQuantity": float,
    "ProductionUOM": str,
    "SalesQuantity": float,
    "SalesUOM": str,
    "SalesValue": float,
    "OpeningQuantity": float,
    "OpeningUOM": str,
    "OpeningValue": float,
    "ClosingQuantity": float,
    "ClosingUOM": str,
    "ClosingValue": float,
    "BoughtOutQuantity": float,
    "BroughtOutUOM": str,
    "ProductMix": float,
    "ModifiedDate": pd.to_datetime,
},
"QuarterlyEPS": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "Sales": float,
    "Expenses": float,
    "EBIDTA": float,
    "Interest": float,
    "Depreciation": float,
    "Extraordinary": float,
    "OPM": float,
    "Tax": float,
    "PATRAW": float,
    "PAT": float,
    "Equity": float,
    "Reserves": float,
    "EPS": float,
    "NPM": float,
    "Ext_Flag": bool,
    "Q1 EPS Growth": float,
    "Q1 Sales Growth": float,
    "Q2 EPS": float,
    "Q2 EPS Growth": float,
    "Q2 Sales": float,
    "Q2 Sales Growth": float,
},
"QuarterlyEPS_DUPLICATED": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "Sales": float,
    "Expenses": float,
    "EBIDTA": float,
    "Interest": float,
    "Depreciation": float,
    "Extraordinary": float,
    "OPM": float,
    "Tax": float,
    "PATRAW": float,
    "PAT": float,
    "Equity": float,
    "Reserves": float,
    "EPS": float,
    "NPM": float,
    "Ext_Flag": bool,
    "Q1 EPS Growth": float,
    "Q1 Sales Growth": float,
    "Q2 EPS": float,
    "Q2 EPS Growth": float,
    "Q2 Sales": float,
    "Q2 Sales Growth": float,
},
"QuarterlyEPS_ToBeDeleted": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "Sales": float,
    "Expenses": float,
    "EBIDTA": float,
    "Interest": float,
    "Depreciation": float,
    "Extraordinary": float,
    "OPM": float,
    "Tax": float,
    "PATRAW": float,
    "PAT": float,
    "Equity": float,
    "Reserves": float,
    "EPS": float,
    "NPM": float,
    "Ext_Flag": bool,
    "Q1 EPS Growth": float,
    "Q1 Sales Growth": float,
    "Q2 EPS": float,
    "Q2 EPS Growth": float,
    "Q2 Sales": float,
    "Q2 Sales Growth": float,
},
"QuarterlyResults": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "OperatingIncome": float,
    "OtherOperatingIncome": float,
    "TotalIncomeFromOperations": float,
    "IntOrDiscOnAdvOrBills": float,
    "IncomeOnInvestments": float,
    "IntOnBalancesWithRBI": float,
    "Others": float,
    "OtherRecurringIncome": float,
    "StockAdjustment": float,
    "RawMaterialConsumed": float,
    "PurchaseOfTradedGoods": float,
    "PowerAndFuel": float,
    "EmployeeExpenses": float,
    "Excise": float,
    "AdminAndSellingExpenses": float,
    "ResearchAndDevelopmentExpenses": float,
    "ExpensesCapitalised": float,
    "OtherExpeses": float,
    "PL_Before_OtherInc_Int_Excpltem_Tax": float,
    "PL_Before_Int_ExcpItem_Tax": float,
    "InterestCharges": float,
    "PL_Before_ExcpItem_Tax": float,
    "ExceptionalItems": float,
    "Depreciation": float,
    "OperatingProfitBeforeProvisionsAndContingencies": float,
    "BankProvisionsMade": float,
    "PL_Before_Tax": float,
    "TaxCharges": float,
    "PL_After_TaxFromOrdineryActivities": float,
    "ExtraOrdinaryItem": float,
    "ReportedPAT": float,
    "PrioryearAdj": float,
    "ReservesWrittenBack": float,
    "EquityCapital": float,
    "ReservesAndSurplus": float,
    "EqDividendRate": float,
    "AggregateOfNonPromotoNoOfShares": float,
    "AggregateOfNonPromotoHoldingPercent": float,
    "GovernmentShare": float,
    "CapitalAdequacyRatio": float,
    "CapitalAdequacyBasell": float,
    "GrossNPA": float,
    "NetNPA": float,
    "Percentage_OfGrossNPA": float,
    "Percentage_OfNetNPA": float,
    "ReturnOnAssets_Per": float,
    "BeforeBasicEPS": float,
    "BeforeDilutedEPS": float,
    "AfterBasicEPS": float,
    "AfterDilutedEPS": float,
    "En_NumberOfShares": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "NonEn_NumberOfShares": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "ModifiedDate": pd.to_datetime,
},
"QuarterlyResults_DUPLICATED": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "OperatingIncome": float,
    "OtherOperatingIncome": float,
    "TotalIncomeFromOperations": float,
    "IntOrDiscOnAdvOrBills": float,
    "IncomeOnInvestments": float,
    "IntOnBalancesWithRBI": float,
    "Others": float,
    "OtherRecurringIncome": float,
    "StockAdjustment": float,
    "RawMaterialConsumed": float,
    "PurchaseOfTradedGoods": float,
    "PowerAndFuel": float,
    "EmployeeExpenses": float,
    "Excise": float,
    "AdminAndSellingExpenses": float,
    "ResearchAndDevelopmentExpenses": float,
    "ExpensesCapitalised": float,
    "OtherExpeses": float,
    "PL_Before_OtherInc_Int_Excpltem_Tax": float,
    "PL_Before_Int_ExcpItem_Tax": float,
    "InterestCharges": float,
    "PL_Before_ExcpItem_Tax": float,
    "ExceptionalItems": float,
    "Depreciation": float,
    "OperatingProfitBeforeProvisionsAndContingencies": float,
    "BankProvisionsMade": float,
    "PL_Before_Tax": float,
    "TaxCharges": float,
    "PL_After_TaxFromOrdineryActivities": float,
    "ExtraOrdinaryItem": float,
    "ReportedPAT": float,
    "PrioryearAdj": float,
    "ReservesWrittenBack": float,
    "EquityCapital": float,
    "ReservesAndSurplus": float,
    "EqDividendRate": float,
    "AggregateOfNonPromotoNoOfShares": float,
    "AggregateOfNonPromotoHoldingPercent": float,
    "GovernmentShare": float,
    "CapitalAdequacyRatio": float,
    "CapitalAdequacyBasell": float,
    "GrossNPA": float,
    "NetNPA": float,
    "Percentage_OfGrossNPA": float,
    "Percentage_OfNetNPA": float,
    "ReturnOnAssets_Per": float,
    "BeforeBasicEPS": float,
    "BeforeDilutedEPS": float,
    "AfterBasicEPS": float,
    "AfterDilutedEPS": float,
    "En_NumberOfShares": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "NonEn_NumberOfShares": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "ModifiedDate": pd.to_datetime,
},
"QuarterlyResults_TEMP": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "OperatingIncome": float,
    "OtherOperatingIncome": float,
    "TotalIncomeFromOperations": float,
    "IntOrDiscOnAdvOrBills": float,
    "IncomeOnInvestments": float,
    "IntOnBalancesWithRBI": float,
    "Others": float,
    "OtherRecurringIncome": float,
    "StockAdjustment": float,
    "RawMaterialConsumed": float,
    "PurchaseOfTradedGoods": float,
    "PowerAndFuel": float,
    "EmployeeExpenses": float,
    "Excise": float,
    "AdminAndSellingExpenses": float,
    "ResearchAndDevelopmentExpenses": float,
    "ExpensesCapitalised": float,
    "OtherExpeses": float,
    "PL_Before_OtherInc_Int_Excpltem_Tax": float,
    "PL_Before_Int_ExcpItem_Tax": float,
    "InterestCharges": float,
    "PL_Before_ExcpItem_Tax": float,
    "ExceptionalItems": float,
    "Depreciation": float,
    "OperatingProfitBeforeProvisionsAndContingencies": float,
    "BankProvisionsMade": float,
    "PL_Before_Tax": float,
    "TaxCharges": float,
    "PL_After_TaxFromOrdineryActivities": float,
    "ExtraOrdinaryItem": float,
    "ReportedPAT": float,
    "PrioryearAdj": float,
    "ReservesWrittenBack": float,
    "EquityCapital": float,
    "ReservesAndSurplus": float,
    "EqDividendRate": float,
    "AggregateOfNonPromotoNoOfShares": float,
    "AggregateOfNonPromotoHoldingPercent": float,
    "GovernmentShare": float,
    "CapitalAdequacyRatio": float,
    "CapitalAdequacyBasell": float,
    "GrossNPA": float,
    "NetNPA": float,
    "Percentage_OfGrossNPA": float,
    "Percentage_OfNetNPA": float,
    "ReturnOnAssets_Per": float,
    "BeforeBasicEPS": float,
    "BeforeDilutedEPS": float,
    "AfterBasicEPS": float,
    "AfterDilutedEPS": float,
    "En_NumberOfShares": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "NonEn_NumberOfShares": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "ModifiedDate": pd.to_datetime,
},
"RatiosBankingVI": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Type": str,
    "FaceValue": float,
    "BasicEPS": float,
    "DilutedEPS": float,
    "CashEPS": float,
    "BVPerShareExclRevalReserve": float,
    "BVPerShareInclRevalReserve": float,
    "DividendPerShare": float,
    "OperatingRevenuePerShare": float,
    "NPPerShare": float,
    "InterestIncomePerEmployee": float,
    "NPPerEmployee": float,
    "BusinessPerEmployee": float,
    "InterestIncomePerBranch": float,
    "NPPerBranches": float,
    "BusinessPerBranches": float,
    "NPM": float,
    "OPM": float,
    "ROA": float,
    "ROE": float,
    "NIM": float,
    "CostToIncome": float,
    "InterestIncomeByEarningAssets": float,
    "NonInterestIncomeByEarningAssets": float,
    "OperatingProfitByEarningAssets": float,
    "OperatingExpensesByEarningAssets": float,
    "InterestExpensesByEarningAssets": float,
    "EnterpriseValue": float,
    "EVPerNetSales": float,
    "PriceToBV": float,
    "PriceToSales": float,
    "RetentionRatios": float,
    "EarningsYield": float,
    "ModifiedDate": pd.to_datetime,
},
"RatiosConslidatedNonBankingVI": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Type": str,
    "FaceValue": float,
    "BasicEPS": float,
    "DilutedEPS": float,
    "CashEPS": float,
    "BVPerShareExclRevalReserve": float,
    "BVPerShareInclRevalReserve": float,
    "OperatingRevenuePerShare": float,
    "PBDITPerShare": float,
    "PBITPerShare": float,
    "PBTPerShare": float,
    "NPPerShare": float,
    "NPAfterMIPerShare": float,
    "PBDITMargin": float,
    "PBITMargin": float,
    "PBTMargin": float,
    "NPMargin": float,
    "NPAfterMIMargin": float,
    "RONW": float,
    "ROCE": float,
    "ReturnOnAssets": float,
    "LongTermDebtEquity": float,
    "DebtEquity": float,
    "AssetTurnover": float,
    "CurrentRatio": float,
    "QuickRatio": float,
    "InventoryTurnoverRatio": float,
    "DividendPayoutNP": float,
    "DividendPayoutCP": float,
    "EarningRetention": float,
    "CashEarningRetention": float,
    "InterestCoverage": float,
    "InterestCoveragePostTax": float,
    "EnterpriseValue": float,
    "EVPerNetSales": float,
    "EVPerEBITDA": float,
    "MarketCapPerSales": float,
    "RetentionRatios": float,
    "PriceToBV": float,
    "PriceToSales": float,
    "EarningsYield": float,
    "ModifiedDate": pd.to_datetime,
},
"RatiosConsolidatedBankingVI": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Type": str,
    "FaceValue": float,
    "BasicEPS": float,
    "DilutedEPS": float,
    "CashEPS": float,
    "BVPerShareExclRevalReserve": float,
    "BVPerShareInclRevalReserve": float,
    "DividendPerShare": float,
    "OperatingRevenuePerShare": float,
    "NPPerShare": float,
    "NPAfterMIPerShare": float,
    "InterestIncomePerEmployee": float,
    "NPPerEmployee": float,
    "NPAfterMIPerEmployee": float,
    "BusinessPerEmployee": float,
    "InterestIncomePerBranch": float,
    "NPPerBranches": float,
    "NPAfterMIPerBranches": float,
    "BusinessPerBranches": float,
    "NPM": float,
    "NPMAfterMI": float,
    "OPM": float,
    "ROA": float,
    "ROE": float,
    "NIM": float,
    "CostToIncome": float,
    "InterestIncomeByEarningAssets": float,
    "NonInterestIncomeByEarningAssets": float,
    "OperatingProfitByEarningAssets": float,
    "OperatingExpensesByEarningAssets": float,
    "InterestExpensesByEarningAssets": float,
    "EnterpriseValue": float,
    "EVPerNetSales": float,
    "PriceToBV": float,
    "PriceToSales": float,
    "RetentionRatios": float,
    "EarningsYield": float,
    "ModifiedDate": pd.to_datetime,
},
"RatiosMergeList": {
    "CompanyCode": float,
    "CompanyName": str,
    "Industry": str,
    "ROEYearEnding": pd.to_datetime,
    "Months": int,
    "FaceValue": float,
    "SalesGrowth": float,
    "NPM": float,
    "ROE": float,
    "Debt": float,
    "TTMYearEnding": pd.to_datetime,
    "GenDate": pd.to_datetime,
},
"RatiosMergeList_DUPLICATED": {
    "CompanyCode": float,
    "CompanyName": str,
    "Industry": str,
    "ROEYearEnding": pd.to_datetime,
    "Months": int,
    "FaceValue": float,
    "SalesGrowth": float,
    "NPM": float,
    "ROE": float,
    "Debt": float,
    "TTMYearEnding": pd.to_datetime,
    "GenDate": pd.to_datetime,
},
"RatiosNonBankingVI": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Type": str,
    "FaceValue": float,
    "BasicEPS": float,
    "DilutedEPS": float,
    "CashEPS": float,
    "BVPerShareExclRevalReserve": float,
    "BVPerShareInclRevalReserve": float,
    "DividendPerShare": float,
    "OperatingRevenuePerShare": float,
    "PBDITPerShare": float,
    "PBITPerShare": float,
    "PBTPerShare": float,
    "NPperShare": float,
    "PBDITMargin": float,
    "PBITMargin": float,
    "PBTMargin": float,
    "NPMargin": float,
    "RONW": float,
    "ROCE": float,
    "ReturnOnAssets": float,
    "LongTermDebtEquity": float,
    "DebtEquity": float,
    "AssetTurnover": float,
    "CurrentRatio": float,
    "QuickRatio": float,
    "InventoryTurnoverRatio": float,
    "DividendPayoutNP": float,
    "DividendPayoutCP": float,
    "EarningRetention": float,
    "CashEarningRetention": float,
    "InterestCoverage": float,
    "InterestCoveragePostTax": float,
    "EnterpriseValue": float,
    "EVPerNetSales": float,
    "EVPerEBITDA": float,
    "MarketCapPerSales": float,
    "RetentionRatios": float,
    "PriceToBV": float,
    "PriceToSales": float,
    "EarningsYield": float,
    "ModifiedDate": pd.to_datetime,
},
"RawMaterials": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "ProductCode": float,
    "ProductName": str,
    "RawMatUOM": str,
    "RawMatQuantity": float,
    "RawMatValue": float,
    "ProdMix": float,
    "ModifiedDate": pd.to_datetime,
},
"Registrars": {
    "CompanyCode": float,
    "AgencyName": str,
    "Address": str,
    "Address2": str,
    "Address3": str,
    "Address4": str,
    "CityName": str,
    "StateName": str,
    "PinCode": str,
    "TelephoneNumbers": str,
    "Phone2": str,
    "Phone3": str,
    "Phone4": str,
    "Fax": str,
    "Telex": str,
    "Grams": str,
    "EmailId": str,
    "Website": str,
    "ModifiedDate": pd.to_datetime,
},
"Rights": {
    "CompanyCode": float,
    "DateOfAnnouncement": pd.to_datetime,
    "ExistingInstrumentType": int,
    "ExistingInstrumentName": str,
    "RatioExisting": float,
    "RatioOffering": float,
    "OfferredInstrumentType": int,
    "OfferredInstrumentName": str,
    "FaceValueExistingInstrument": float,
    "FaceValueOfferedInstrument": float,
    "RightsPremium": float,
    "RecordDate": pd.to_datetime,
    "BookClosureStartDate": pd.to_datetime,
    "BookClosureEndDate": pd.to_datetime,
    "XRDate": pd.to_datetime,
    "Remarks": str,
    "DeleteFlag": bool,
    "ModifiedDate": pd.to_datetime,
},
"SchemeBoardOfAMC": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "TitleDescription": str,
    "PersonName": str,
    "DesignationDescription": str,
    "ModifyDate": pd.to_datetime,
    "DeleteFlag": bool,
    "RowId": int,
},
"SchemeBoardOfTrustees": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "TitleDescription": str,
    "PersonName": str,
    "DesignationDescription": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
    "RowId": int,
},
"SchemeBonusDetails": {
    "SchemeCode": float,
    "SchemePlanCode": float,
    "BonusDate": pd.to_datetime,
    "Ratio_Offered": str,
    "Ratio_Existing": str,
    "Remarks": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
    "RowId": int,
},
"SchemeCategoryDetails": {
    "SchemeClassCode": float,
    "SchemeClassDescription": str,
    "CategoryNavDate": pd.to_datetime,
    "NoOfSchemes": int,
    "OneDayReturn": float,
    "OneWeekReturn": float,
    "TwoWeeksReturn": float,
    "OneMonthReturn": float,
    "ThreeMonthsReturn": float,
    "SixMonthsReturn": float,
    "NineMonthsReturn": float,
    "OneYearReturn": float,
    "TwoYearReturn": float,
    "ThreeYearReturn": float,
    "FiveYearReturn": float,
    "InceptionReturn": float,
    "ModifiedDate": pd.to_datetime,
},
"SchemeCorporateDividendDetails": {
    "SchemeCode": float,
    "SchemePlanCode": int,
    "DividendDate": pd.to_datetime,
    "Percentage": float,
    "DividendPerUnit": float,
    "DividendTypeDescription": str,
    "Remarks": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
    "RowId": int,
},
"SchemeDividendDetails": {
    "SchemeCode": float,
    "SchemePlanCode": float,
    "DividendTypeDescription": str,
    "DividendDate": pd.to_datetime,
    "Percentage": float,
    "DividendPerUnit": float,
    "Remarks": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
    "RowId": int,
},
"SchemeFundManagerProfile": {
    "SchemeCode": float,
    "SchemePlanCode": float,
    "FundManagerName": str,
    "FMProfile": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
},
"SchemeLocations": {
    "EntityCode": float,
    "LocationType": str,
    "Address": str,
    "CityName": str,
    "StateName": str,
    "CountryName": str,
    "Pincode": str,
    "TelephoneNumbers": str,
    "Telex": str,
    "Fax": str,
    "Grams": str,
    "EMailID": str,
    "WebSite": str,
    "ModifiedDate": pd.to_datetime,
},
"SchemeMaster": {
    "SchemeCode": float,
    "SchemeName": str,
    "CompanyCode": float,
    "AMCCode": float,
    "SchemeType": int,
    "SchemeTypeDescription": str,
    "SchemePlanCode": float,
    "SchemePlanDescription": str,
    "SchemeClassCode": int,
    "SchemeClassDescription": str,
    "SchemeCategoryCode": int,
    "SchemeCategoryDescription": str,
    "MainCategory": str,
    "SubCategory": str,
    "LaunchDate": pd.to_datetime,
    "OpenDate": pd.to_datetime,
    "CloseDate": pd.to_datetime,
    "RedemptionDate": pd.to_datetime,
    "MinInvAmnt": float,
    "IntPrice": float,
    "IntPricUOM": int,
    "IntPUOMDes": str,
    "Size": float,
    "SizeUOM": float,
    "SizeUOMDescription": str,
    "OfferPrice": float,
    "AmountRaised": float,
    "AmountRaisedUOM": float,
    "AmountRaisedUOMDescription": str,
    "FundManagerPrefix": str,
    "FundManagerName": str,
    "ListingInformation": bool,
    "EntryLoad": str,
    "ExitLoad": str,
    "RedemptionFerq": str,
    "ISIN_Div_Payout_ISIN_Growth": str,
    "ISIN_Div_Reinvestment": str,
    "RegistrarName": str,
    "EntryLoad_WithSlab_Desc": str,
    "ExitLoad_WithSlab_Desc": str,
    "SIP_Min_Invt": str,
    "SIPEntryLoad": str,
    "SIPExitLoad": str,
    "ExpensesRatio": str,
    "PortFolioTurnOverRatio": str,
    "SWP": str,
    "STP": str,
    "RiskoMeter": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
},
"SchemeMasterCopy": {
    "SchemeCode": float,
    "SchemeName": str,
    "CompanyCode": float,
    "AMCCode": float,
    "SchemeType": int,
    "SchemeTypeDescription": str,
    "SchemePlanCode": float,
    "SchemePlanDescription": str,
    "SchemeClassCode": int,
    "SchemeClassDescription": str,
    "SchemeCategoryCode": int,
    "SchemeCategoryDescription": str,
    "MainCategory": str,
    "SubCategory": str,
    "LaunchDate": pd.to_datetime,
    "OpenDate": pd.to_datetime,
    "CloseDate": pd.to_datetime,
    "RedemptionDate": pd.to_datetime,
    "MinInvAmnt": float,
    "IntPrice": float,
    "IntPricUOM": int,
    "IntPUOMDes": str,
    "Size": float,
    "SizeUOM": float,
    "SizeUOMDescription": str,
    "OfferPrice": float,
    "AmountRaised": float,
    "AmountRaisedUOM": float,
    "AmountRaisedUOMDescription": str,
    "FundManagerPrefix": str,
    "FundManagerName": str,
    "ListingInformation": bool,
    "EntryLoad": str,
    "ExitLoad": str,
    "RedemptionFerq": str,
    "ISIN_Div_Payout_ISIN_Growth": str,
    "ISIN_Div_Reinvestment": str,
    "RegistrarName": str,
    "EntryLoad_WithSlab_Desc": str,
    "ExitLoad_WithSlab_Desc": str,
    "SIP_Min_Invt": str,
    "SIPEntryLoad": str,
    "SIPExitLoad": str,
    "ExpensesRatio": str,
    "PortFolioTurnOverRatio": str,
    "SWP": str,
    "STP": str,
    "RiskoMeter": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
},
"SchemeMaster_Join_DUPLICATED": {
    "SchemeCode": float,
    "SchemeName": str,
    "CompanyCode": float,
    "AMCCode": float,
    "SchemeType": int,
    "SchemeTypeDescription": str,
    "SchemePlanCode": float,
    "SchemePlanDescription": str,
    "SchemeClassCode": int,
    "SchemeClassDescription": str,
    "SchemeCategoryCode": int,
    "SchemeCategoryDescription": str,
    "MainCategory": str,
    "SubCategory": str,
    "LaunchDate": pd.to_datetime,
    "OpenDate": pd.to_datetime,
    "CloseDate": pd.to_datetime,
    "RedemptionDate": pd.to_datetime,
    "MinInvAmnt": float,
    "IntPrice": float,
    "IntPricUOM": int,
    "IntPUOMDes": str,
    "Size": float,
    "SizeUOM": float,
    "SizeUOMDescription": str,
    "OfferPrice": float,
    "AmountRaised": float,
    "AmountRaisedUOM": float,
    "AmountRaisedUOMDescription": str,
    "FundManagerPrefix": str,
    "FundManagerName": str,
    "ListingInformation": bool,
    "EntryLoad": str,
    "ExitLoad": str,
    "RedemptionFerq": str,
    "ISIN_Div_Payout_ISIN_Growth": str,
    "ISIN_Div_Reinvestment": str,
    "RegistrarName": str,
    "EntryLoad_WithSlab_Desc": str,
    "ExitLoad_WithSlab_Desc": str,
    "SIP_Min_Invt": str,
    "SIPEntryLoad": str,
    "SIPExitLoad": str,
    "ExpensesRatio": str,
    "PortFolioTurnOverRatio": str,
    "SWP": str,
    "STP": str,
    "RiskoMeter": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
},
"SchemeMaster_OLD": {
    "SchemeCode": float,
    "SchemeName": str,
    "CompanyCode": float,
    "AMCCode": float,
    "SchemeType": int,
    "SchemeTypeDescription": str,
    "SchemePlanCode": float,
    "SchemePlanDescription": str,
    "SchemeClassCode": int,
    "SchemeClassDescription": str,
    "SchemeCategoryCode": int,
    "SchemeCategoryDescription": str,
    "MainCategory": str,
    "SubCategory": str,
    "LaunchDate": pd.to_datetime,
    "OpenDate": pd.to_datetime,
    "CloseDate": pd.to_datetime,
    "RedemptionDate": pd.to_datetime,
    "MinInvAmnt": float,
    "IntPrice": float,
    "IntPricUOM": int,
    "IntPUOMDes": str,
    "Size": float,
    "SizeUOM": float,
    "SizeUOMDescription": str,
    "OfferPrice": float,
    "AmountRaised": float,
    "AmountRaisedUOM": float,
    "AmountRaisedUOMDescription": str,
    "FundManagerPrefix": str,
    "FundManagerName": str,
    "ListingInformation": bool,
    "EntryLoad": str,
    "ExitLoad": str,
    "RedemptionFerq": str,
    "ISIN_Div_Payout_ISIN_Growth": str,
    "ISIN_Div_Reinvestment": str,
    "RegistrarName": str,
    "EntryLoad_WithSlab_Desc": str,
    "ExitLoad_WithSlab_Desc": str,
    "SIP_Min_Invt": str,
    "SIPEntryLoad": str,
    "SIPExitLoad": str,
    "ExpensesRatio": str,
    "PortFolioTurnOverRatio": str,
    "SWP": str,
    "STP": str,
    "RiskoMeter": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
},
"SchemeNAVCurrentPrices": {
    "SecurityCode": float,
    "Ticker": str,
    "DateTime": pd.to_datetime,
    "NAVAmount": float,
    "RepurchaseLoad": float,
    "RepurchasePrice": float,
    "SaleLoad": float,
    "SalePrice": float,
    "PrevNAVAmount": float,
    "PrevRepurchasePrice": float,
    "PrevSalePrice": float,
    "PercentageChange": float,
    "Prev1WeekAmount": float,
    "Prev1WeekPer": float,
    "Prev1MonthAmount": float,
    "Prev1MonthPer": float,
    "Prev3MonthsAmount": float,
    "Prev3MonthsPer": float,
    "Prev6MonthsAmount": float,
    "Prev6MonthsPer": float,
    "Prev9MonthsAmount": float,
    "Prev9MonthsPer": float,
    "PrevYearAmount": float,
    "PrevYearPer": float,
    "Prev2YearAmount": float,
    "Prev2YearPer": float,
    "Prev2YearCompPer": float,
    "Prev3YearAmount": float,
    "Prev3YearPer": float,
    "Prev3YearCompPer": float,
    "Prev5YearAmount": float,
    "Prev5YearPer": float,
    "Prev5YearCompPer": float,
    "ListDate": pd.to_datetime,
    "ListAmount": float,
    "ListPer": float,
    "ListCompPer": float,
    "FiftyTwoWeekHighNAV": float,
    "FiftyTwoWeekLowNAV": float,
    "FiftyTwoWeekHighNAVDate": pd.to_datetime,
    "FiftyTwoWeekLowNAVDate": pd.to_datetime,
    "Rank": int,
},
"SchemeNAVDetails": {
    "SecurityCode": str,
    "NAVDate": pd.to_datetime,
    "NAVAmount": float,
    "RepurchaseLoad": float,
    "RepurchasePrice": float,
    "SaleLoad": float,
    "SalePrice": float,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
},
"SchemeNAVMaster": {
    "SchemeCode": float,
    "SecurityCode": float,
    "SchemePlanCode": int,
    "SchemePlanDescription": str,
    "MappingCode": str,
    "MappingName": str,
    "IssuePrice": float,
    "Description": str,
    "IssueDate": pd.to_datetime,
    "ExpiryDate": pd.to_datetime,
    "FaceValue": float,
    "MarketLot": float,
    "ISINCode": str,
    "BenchMarkIndex": float,
    "BenchMarkIndexName": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
},
"SchemeObjectives": {
    "SchemeCode": float,
    "SchemePlanCode": int,
    "Objectives": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
},
"SchemePortfolioHeader": {
    "SchemeCode": float,
    "SchemePlanCode": int,
    "HoldingDate": pd.to_datetime,
    "TotalMarketValue": float,
    "ModifiedDate": pd.to_datetime,
},
"SchemeRegistrars": {
    "CompanyCode": float,
    "AgencyName": str,
    "Address": str,
    "Address2": str,
    "Address3": str,
    "Address4": str,
    "CityName": str,
    "StateName": str,
    "PinCode": str,
    "TelephoneNumbers": str,
    "Phone2": str,
    "Phone3": str,
    "Phone4": str,
    "Fax": str,
    "Telex": str,
    "Grams": str,
    "EmailId": str,
    "Website": str,
    "ModifiedDate": pd.to_datetime,
},
"SchemeSplitsDetails": {
    "SchemeCode": float,
    "SchemePlanCode": int,
    "SplitsDate": pd.to_datetime,
    "Old_FaceValue": float,
    "New_FaceValue": float,
    "Remarks": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
},
"SchemeTaxBenefit": {
    "SchemeCode": float,
    "SchemePlanCode": float,
    "SchemePlanDescription": str,
    "ITActSection": str,
    "TaxBenefits": str,
    "ModifyDate": pd.to_datetime,
    "DeleteFlag": bool,
    "RowId": int,
},
"SchemewisePortfolio": {
    "SchemeCode": float,
    "SchemePlanCode": int,
    "HoldingDate": pd.to_datetime,
    "InvestedCompanyCode": float,
    "InvestedCompanyName": str,
    "IndustryCode": int,
    "IndustryName": str,
    "InstrumentName": str,
    "ListingInformation": bool,
    "Quantity": float,
    "Percentage": float,
    "IsItNPA": bool,
    "MarketValue": float,
    "PortfolioUOMDescription": str,
    "Rating": str,
    "ISINCode": str,
    "DeleteFlag": bool,
    "ModifiedDate": pd.to_datetime,
    "RowId": int,
},
"SectorDivisor": {
    "IndexName": str,
    "IndexValue": float,
    "Divisor": float,
    "Date": pd.to_datetime,
    "OS": float,
},
"SectorDivisor_DUPLICATED": {
    "IndexName": str,
    "IndexValue": float,
    "Divisor": float,
    "Date": pd.to_datetime,
    "OS": float,
},
"SectorIndexList": {
    "GenDate": pd.to_datetime,
    "SectorIndexName": str,
    "Sector": str,
    "Open": float,
    "High": float,
    "Low": float,
    "Close": float,
    "Volume": float,
    "PE": float,
    "EPS": float,
    "CompanyCount": int,
    "OS": float,
    "Earnings Growth": float,
},
"SectorIndexList_DUPLICATED": {
    "GenDate": pd.to_datetime,
    "SectorIndexName": str,
    "Sector": str,
    "Open": float,
    "High": float,
    "Low": float,
    "Close": float,
    "Volume": float,
    "PE": float,
    "EPS": float,
    "CompanyCount": int,
    "OS": float,
    "Earnings Growth": float,
},
"ShareHolding": {
    "CompanyCode": float,
    "SHPDate": pd.to_datetime,
    "Capital": float,
    "FaceValue": float,
    "NoOfShares": float,
    "Promoters": float,
    "Directors": float,
    "SubsidiaryCompanies": float,
    "OtherCompanies": float,
    "ICICI": float,
    "UTI": float,
    "IDBI": float,
    "GenInsuranceComp": float,
    "LifeInsuranceComp": float,
    "StateFinCorps": float,
    "InduFinCorpIndia": float,
    "ForeignNRI": float,
    "ForeignCollaborators": float,
    "ForeignOCB": float,
    "ForeignOthers": float,
    "ForeignInstitutions": float,
    "ForeignIndustries": float,
    "StateGovt": float,
    "CentralGovt": float,
    "GovtCompanies": float,
    "GovtOthers": float,
    "Others": float,
    "NBanksMutualFunds": float,
    "HoldingCompanies": float,
    "GeneralPublic": float,
    "Employees": float,
    "FinancialInstitutions": float,
    "ForeignPromoter": float,
    "GDR": float,
    "PersonActingInConcert": float,
    "Total": float,
    "ModifiedDate": pd.to_datetime,
},
"Splits": {
    "CompanyCode": float,
    "DateOfAnnouncement": pd.to_datetime,
    "OldFaceValue": float,
    "NewFaceValue": float,
    "RecordDate": pd.to_datetime,
    "BookClosureStartDate": pd.to_datetime,
    "BookClosureEndDate": pd.to_datetime,
    "XSDate": pd.to_datetime,
    "Remarks": str,
    "DeleteFlag": bool,
    "ModifiedDate": pd.to_datetime,
},
"SubSectorDivisor": {
    "IndexName": str,
    "IndexValue": float,
    "Divisor": float,
    "Date": pd.to_datetime,
    "OS": float,
},
"SubSectorDivisor_DUPLICATED": {
    "IndexName": str,
    "IndexValue": float,
    "Divisor": float,
    "Date": pd.to_datetime,
    "OS": float,
},
"SubSectorIndexList": {
    "GenDate": pd.to_datetime,
    "SubSectorIndexName": str,
    "SubSector": str,
    "Open": float,
    "High": float,
    "Low": float,
    "Close": float,
    "Volume": float,
    "PE": float,
    "EPS": float,
    "CompanyCount": int,
    "OS": float,
    "Earnings Growth": float,
},
"SubSectorIndexList_DUPLICATED": {
    "GenDate": pd.to_datetime,
    "SubSectorIndexName": str,
    "SubSector": str,
    "Open": float,
    "High": float,
    "Low": float,
    "Close": float,
    "Volume": float,
    "PE": float,
    "EPS": float,
    "CompanyCount": int,
    "OS": float,
    "Earnings Growth": float,
},
"Subsidiaries": {
    "ParentCompanyCode": float,
    "ParentCompanyName": str,
    "CompanyCode": float,
    "CompanyName": str,
    "HoldingPercentage": float,
    "Remarks": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
},
"TTM": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "Sales": float,
    "Expenses": float,
    "EBIDTA": float,
    "Interest": float,
    "Depreciation": float,
    "Extraordinary": float,
    "OPM": float,
    "Tax": float,
    "PAT": float,
    "Equity": float,
    "Reserves": float,
    "EPS": float,
    "NPM": float,
},
"TTM_BACKUP": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "Sales": float,
    "Expenses": float,
    "EBIDTA": float,
    "Interest": float,
    "Depreciation": float,
    "Extraordinary": float,
    "OPM": float,
    "Tax": float,
    "PAT": float,
    "Equity": float,
    "Reserves": float,
    "EPS": float,
    "NPM": float,
},
"TTM_DUPLICATED": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "Sales": float,
    "Expenses": float,
    "EBIDTA": float,
    "Interest": float,
    "Depreciation": float,
    "Extraordinary": float,
    "OPM": float,
    "Tax": float,
    "PAT": float,
    "Equity": float,
    "Reserves": float,
    "EPS": float,
    "NPM": float,
},
"TTM_ONWAIT": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "Sales": float,
    "Expenses": float,
    "EBIDTA": float,
    "Interest": float,
    "Depreciation": float,
    "Extraordinary": float,
    "OPM": float,
    "Tax": float,
    "PAT": float,
    "Equity": float,
    "Reserves": float,
    "EPS": float,
    "NPM": float,
},
"TTM_TEMP": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "Sales": float,
    "Expenses": float,
    "EBIDTA": float,
    "Interest": float,
    "Depreciation": float,
    "Extraordinary": float,
    "OPM": float,
    "Tax": float,
    "PAT": float,
    "Equity": float,
    "Reserves": float,
    "EPS": float,
    "NPM": float,
},
"index_btt_mapping": {
    "IndexName": str,
    "BTTCode": str,
},
"mf_category_mapping": {
    "scheme_code": float,
    "scheme_name": str,
    "scheme_category": str,
    "date": pd.to_datetime,
    "btt_scheme_code": str,
    "btt_scheme_category": str,
},
"mf_category_mapping_DUPLICATED": {
    "scheme_code": float,
    "scheme_name": str,
    "scheme_category": str,
    "date": pd.to_datetime,
    "btt_scheme_code": str,
    "btt_scheme_category": str,
    "Updated": str,
},
"mf_category_mapping_copy": {
    "scheme_code": float,
    "scheme_name": str,
    "scheme_category": str,
    "date": pd.to_datetime,
    "btt_scheme_code": str,
    "btt_scheme_category": str,
},
"mf_ohlc": {
    "date": pd.to_datetime,
    "btt_scheme_code": str,
    "open": float,
    "high": float,
    "low": float,
    "close": float,
    "volume": float,
},
"mf_ohlc_DUPLICATED": {
    "date": pd.to_datetime,
    "btt_scheme_code": str,
    "open": float,
    "high": float,
    "low": float,
    "close": float,
    "volume": float,
},
"nse_index_change": {
    "symbol": str,
    "change": float,
    "date": pd.to_datetime,
},
"nse_index_change_DUPLICATED": {
    "symbol": str,
    "change": float,
    "date": pd.to_datetime,
},
"ohlc_monthly": {
    "company_code": float,
    "company_name": str,
    "nse_code": str,
    "bse_code": int,
    "open": float,
    "high": float,
    "low": float,
    "close": float,
    "volume": float,
    "date": pd.to_datetime,
},
"ohlc_weekly": {
    "company_code": float,
    "open": float,
    "high": float,
    "low": float,
    "close": float,
    "volume": int,
    "date": pd.to_datetime,
},
"ohlc_weekly_DUPLICATED": {
    "company_code": float,
    "open": float,
    "high": float,
    "low": float,
    "close": float,
    "volume": int,
    "date": pd.to_datetime,
},
"temp_scheme_nav_details": {
    "SecurityCode": float,
    "NAVDate": str,
    "NAVAmount": int,
    "RepurchaseLoad": int,
    "RepurchasePrice": int,
    "SaleLoad": int,
    "SalePrice": int,
    "ModifiedDate": int,
    "DeleteFlag": int,
},
"temp_schemewiseportfolio": {
    "SchemeCode": str,
    "SchemePlanCode": str,
    "HoldingDate": str,
    "InvestedCompanyCode": int,
    "InvestedCompanyName": int,
    "IndustryCode": int,
    "IndustryName": int,
    "InstrumentName": int,
    "ListingInformation": int,
    "Quantity": int,
    "Percentage": int,
    "IsItNPA": int,
    "MarketValue": int,
    "PortfolioUOMDescription": int,
    "Rating": int,
    "ISINCode": int,
    "DeleteFlag": int,
    "ModifiedDate": int,
    "RowId": int,
},
"users": {
    "username": str,
    "password": str,
},
"FRS-NAVRank": {
    "SchemeCode": float,
    "SchemeName": str,
    "Date": pd.to_datetime,
    "Current": float,
    "1 Day": float,
    "1 Day Rank": float,
    "1 Week": float,
    "1 Week Rank": float,
    "1 Month": float,
    "1 Month Rank": float,
    "3 Month": float,
    "3 Month Rank": float,
    "6 Month": float,
    "6 Month Rank": float,
    "9 Month": float,
    "9 Month Rank": float,
    "1 Year": float,
    "1 Year Rank": float,
    "2 Year": float,
    "2 Year Rank": float,
    "3 Year": float,
    "3 Year Rank": float,
    "5 Year": float,
    "5 Year Rank": float,
    "AUM": float,
    "Scheme Rank": float,
    "btt_scheme_category": str,
    "btt_scheme_code": str,
},
"NewHighNewLow": {
    "Date": pd.to_datetime,
    "30DNH": int,
    "30DNL": int,
    "30DNHNL": int,
    "90DNH": int,
    "90DNL": int,
    "90DNHNL": int,
    "52WNH": int,
    "52WNL": int,
    "52WNHNL": int,
},
"PRS": {
    "CompanyName": str,
    "NSECode": str,
    "BSECode": int,
    "Open": float,
    "High": float,
    "Low": float,
    "Close": float,
    "Volume": int,
    "Value": float,
    "52 High": float,
    "52 Low": float,
    "52 High Date": pd.to_datetime,
    "52 Low Date": pd.to_datetime,
    "New High 52W": bool,
    "New High 90D": bool,
    "New High 30D": bool,
    "New Low 52W": bool,
    "New Low 90D": bool,
    "New Low 30D": bool,
    "RR 1D": float,
    "RR 5D": float,
    "RR 10D": float,
    "RR 30D": float,
    "RR 60D": float,
    "Change 52W": float,
    "Change 90D": float,
    "Change 30D": float,
    "RR 52W": float,
    "RR 90D": float,
    "RS 52W": float,
    "RS 90D": float,
    "RS 30D": float,
    "Combined Strength": float,
    "ISIN": str,
    "Date": pd.to_datetime,
    "RR30_Replaced": bool,
    "RR60_Replaced": bool,
    "RR90_Replaced": bool,
    "RR52W_Replaced": bool,
    "Off-High": float,
    "Off-Low": float,
    "CompanyCode": float,
    "Value Average": float,
    "Market Cap Value": float,
    "Market Cap Class": str,
    "Market Cap Rank": int,
    "PE": float,
    "PE High": float,
    "PE High Date": pd.to_datetime,
    "PE Low": float,
    "PE Low Date": pd.to_datetime,
},
"SMR": {
    "CompanyCode": float,
    "NSESymbol": str,
    "BSESymbol": int,
    "Industry": str,
    "ROEYearEnding": pd.to_datetime,
    "Months": int,
    "TTMYearEnding": pd.to_datetime,
    "SalesGrowth": float,
    "Sales Growth Rank": float,
    "NPM": float,
    "NPM Rank": float,
    "ROE": float,
    "ROE Rank": float,
    "SMR Grade": str,
    "SMR": float,
    "SMR Rank": float,
    "SMRDate": pd.to_datetime,
    "CompanyName": str,
    "ISIN": str,
},
"Consolidated_EPS": {
    "CompanyCode": float,
    "NSECode": str,
    "BSECode": int,
    "CompanyName": str,
    "ISIN": str,
    "Months": int,
    "Quarter": int,
    "YearEnding": pd.to_datetime,
    "Q1 EPS": float,
    "Q1 EPS Growth": float,
    "Q1 Sales": float,
    "Q1 Sales Growth": float,
    "Q2 EPS": float,
    "Q2 EPS Growth": float,
    "Q2 Sales": float,
    "Q2 Sales Growth": float,
    "TTM1 EPS Growth": float,
    "TTM1 Sales Growth": float,
    "TTM2 EPS Growth": float,
    "TTM2 Sales Growth": float,
    "TTM3 EPS Growth": float,
    "TTM3 Sales Growth": float,
    "NPM": float,
    "EPS Rating": float,
    "Ranking": float,
    "EPSDate": pd.to_datetime,
    "E_ERS": float,
},
"EPS": {
    "CompanyCode": float,
    "NSECode": str,
    "BSECode": int,
    "CompanyName": str,
    "ISIN": str,
    "Months": int,
    "Quarter": int,
    "YearEnding": pd.to_datetime,
    "Q1 EPS": float,
    "Q1 EPS Growth": float,
    "Q1 Sales": float,
    "Q1 Sales Growth": float,
    "Q2 EPS": float,
    "Q2 EPS Growth": float,
    "Q2 Sales": float,
    "Q2 Sales Growth": float,
    "TTM1 EPS Growth": float,
    "TTM1 Sales Growth": float,
    "TTM2 EPS Growth": float,
    "TTM2 Sales Growth": float,
    "TTM3 EPS Growth": float,
    "TTM3 Sales Growth": float,
    "NPM": float,
    "EPS Rating": float,
    "Ranking": float,
    "EPSDate": pd.to_datetime,
    "E_ERS": float,
},
"STANDALONE_EPS": {
    "CompanyCode": float,
    "NSECode": str,
    "BSECode": int,
    "CompanyName": str,
    "ISIN": str,
    "Months": int,
    "Quarter": int,
    "YearEnding": pd.to_datetime,
    "Q1 EPS": float,
    "Q1 EPS Growth": float,
    "Q1 Sales": float,
    "Q1 Sales Growth": float,
    "Q2 EPS": float,
    "Q2 EPS Growth": float,
    "Q2 Sales": float,
    "Q2 Sales Growth": float,
    "TTM1 EPS Growth": float,
    "TTM1 Sales Growth": float,
    "TTM2 EPS Growth": float,
    "TTM2 Sales Growth": float,
    "TTM3 EPS Growth": float,
    "TTM3 Sales Growth": float,
    "NPM": float,
    "EPS Rating": float,
    "Ranking": float,
    "EPSDate": pd.to_datetime,
    "E_ERS": float,
},
"IRS": {
    "GenDate": pd.to_datetime,
    "IndexName": str,
    "Index": str,
    "Open": float,
    "High": float,
    "Low": float,
    "Close": float,
    "MCap_Open_Index": float,
    "MCap_High_Index": float,
    "MCap_Low_Index": float,
    "MCap_Close_Index": float,
    "Volume": float,
    "PE": float,
    "EPS": float,
    "CompanyCount": int,
    "OS": float,
    "Rank": float,
    "Change": float,
    "Earnings Growth": float,
    "ff_open_sum": float,
    "ff_high_sum": float,
    "ff_low_sum": float,
    "ff_close_sum": float,
    "MCap_Open_sum": float,
    "MCap_High_sum": float,
    "MCap_Low_sum": float,
    "MCap_Close_sum": float,
    "PE High": float,
    "PE High Date": pd.to_datetime,
    "PE Low": float,
    "PE Low Date": pd.to_datetime,
}
}

# Create a connection to the database
conn = psycopg2.connect(
    host="localhost",
    database="BravisaDB",
    user="postgres",
    password="edsols" 
)

create_table_queries = []

# Create a cursor object
cur = conn.cursor()

# Get the list of all schemas excluding temporary and system schemas
schema_sql = """
SELECT schema_name 
FROM information_schema.schemata
WHERE schema_name NOT LIKE 'pg_temp_%'
AND schema_name NOT LIKE 'pg_toast%'
AND schema_name NOT LIKE 'information%'
AND schema_name NOT LIKE 'pg_catalog%'
"""
cur.execute(schema_sql)
schemas = cur.fetchall()

# Get the CREATE TABLE statements for all tables in each schema
for schema in schemas:
    if schema[0] not in ['mf_analysis', 'dash_process']:
        schema_name = schema[0]
        table_sql = f"""
        SELECT table_name 
        FROM information_schema.tables
        WHERE table_schema = '{schema_name}'
        """
        cur.execute(table_sql)
        tables = cur.fetchall()

        date_columns_collection = []
        for table in tables:
            table_name = table[0]
            # print(table_name)   

            # get the expected_data_types for the table
            columns = expected_data_types[table_name]

            # columns with pd.to_datetime 
            date_columns = [k for k, v in columns.items() if v == pd.to_datetime]
            
            # collect all the date_columns in a list
            date_columns_collection = date_columns_collection + date_columns
        
        #print all the unique values and their occurence in descending order from dates_columns_collection
        print(Counter(date_columns_collection).most_common())

                

In [ ]:
import os
import zipfile

# location of the backup zip files
Ziplocation = "G:\\FB Files Backup\\"

fbfiles = os.getcwd() + "\\FBFiles\\"

print(fbfiles)

for folder in os.listdir(Ziplocation):
    if folder not in ['BTT Data (FB Files Full Backup)', 'drive-download', 'Missing Result', 'ALLFBFiles', 'OHLC files']:
        print(folder)
        for file in os.listdir(os.path.join(Ziplocation, folder)):
            if not file.endswith(('.zip', '.csv')):
                print(file)
                for fbfile in os.listdir(os.path.join(Ziplocation, folder, file)):
                    if fbfile.endswith('.zip') and fbfile.startswith('FB'):
                        print(fbfile)
                        # unzip fbfile into fbfiles folder as a folder with same name
                        with zipfile.ZipFile(os.path.join(Ziplocation, folder, file, fbfile), 'r') as zip_ref:
                            zip_ref.extractall(os.path.join(fbfiles, fbfile.split('.zip')[0]))
                            print('Extracted', fbfile)



e:\BravisaProject\app\FBFiles\
2020
2021
2022
2023
2024
Apr-24
Aug-24
Feb-24
Jan-24
July-24
Jun-24
Mar-24
May-24
Nov-24
FB0111202401.zip
Extracted FB0111202401.zip
FB0111202402.zip
Extracted FB0111202402.zip
FB0111202403.zip
Extracted FB0111202403.zip
FB0211202401.zip
Extracted FB0211202401.zip
FB0211202402.zip
Extracted FB0211202402.zip
FB0211202403.zip
Extracted FB0211202403.zip
FB0411202401.zip
Extracted FB0411202401.zip
FB0411202402.zip
Extracted FB0411202402.zip
FB0411202403.zip
Extracted FB0411202403.zip
FB0511202401.zip
Extracted FB0511202401.zip
FB0511202402.zip
Extracted FB0511202402.zip
FB0511202403.zip
Extracted FB0511202403.zip
FB0611202401.zip
Extracted FB0611202401.zip
FB0611202402.zip
Extracted FB0611202402.zip
FB0611202403.zip
Extracted FB0611202403.zip
FB0711202401.zip
Extracted FB0711202401.zip
FB0711202402.zip
Extracted FB0711202402.zip
FB0711202403.zip
Extracted FB0711202403.zip
FB0811202401.zip
Extracted FB0811202401.zip
FB0811202402.zip
Extracted FB0811202402.zip


In [2]:
import os 
import pandas as pd
import psycopg2
#import timedelta
from datetime import timedelta
from lib import  index_ohlc, split_bonus
from Test_reports import ohlc
#ignore warnings
import warnings
from Test_reports.IRSchecker import IRS

warnings.filterwarnings("ignore")

def delIRS(curr_date, conn, cur):
    sql_IndustryList = """delete from "IndustryList" 
                        where "GenDate" = %s"""
    # execute the query with curr_date
    cur.execute(sql_IndustryList, (curr_date,))
    conn.commit()
    print("Deleted the records from IndustryList table for the date: ", curr_date)
    sql_SectorDivisor = """delete from "SectorDivisor"
                            where "Date" = %s"""
    cur.execute(sql_SectorDivisor, (curr_date,))
    conn.commit()
    print("Deleted the records from SectorDivisor table for the date: ", curr_date)
    sql_SubSecotDivisor = """delete from "SubSectorDivisor" 
                            where "Date" = %s"""
    cur.execute(sql_SubSecotDivisor, (curr_date,))
    conn.commit()
    print("Deleted the records from SubSectorDivisor table for the date: ", curr_date)
    sql_IndustryDivisor = """delete from "IndustryDivisor" 
                            where "Date" = %s"""
    cur.execute(sql_IndustryDivisor, (curr_date,))
    conn.commit()
    print("Deleted the records from IndustryDivisor table for the date: ", curr_date)
    sql_SubIndustryDivisor = """delete from "SubIndustryDivisor" 
                                where "Date" = %s"""
    cur.execute(sql_SubIndustryDivisor, (curr_date,))
    conn.commit()
    print("Deleted the records from SubIndustryDivisor table for the date: ", curr_date)
    sql_SectorIndexList = """delete from "SectorIndexList" 
                            where "GenDate" = %s"""
    cur.execute(sql_SectorIndexList, (curr_date,))
    conn.commit()
    print("Deleted the records from SectorIndexList table for the date: ", curr_date)
    sql_SubSectorIndexList = """delete from "SubSectorIndexList" 
                                where "GenDate" = %s"""
    cur.execute(sql_SubSectorIndexList, (curr_date,))
    conn.commit()
    print("Deleted the records from SubSectorIndexList table for the date: ", curr_date)
    sql_IndustryIndexList = """delete from "IndustryIndexList" 
                                where "GenDate" = %s"""
    cur.execute(sql_IndustryIndexList, (curr_date,))
    conn.commit()
    print("Deleted the records from IndustryIndexList table for the date: ", curr_date)
    sql_SubIndustryIndexList = """delete from "SubIndustryIndexList" 
                                    where "GenDate" = %s""" 
    cur.execute(sql_SubIndustryIndexList, (curr_date,))
    conn.commit()
    print("Deleted the records from SubIndustryIndexList table for the date: ", curr_date)
    sql_IRS = """delete from "Reports"."IRS" 
                where "GenDate" = %s"""
    cur.execute(sql_IRS, (curr_date,))
    conn.commit()
    print("Deleted the records from IRS table for the date: ", curr_date)
    sql_IndexHistory = """delete from "IndexHistory"  
                        where "DATE" = %s """
    cur.execute(sql_IndexHistory, (curr_date,))
    conn.commit() 
    
def insert_shareholding(conn, cur,fbname ):
    """ Insert the ShareHolding data into database

    Operation:
        Set the path for csv file, and fetch the data from ShareHolding.csv file.
        delete the data based on key column CompanyCode and SHPDate,  
        Export data into 'ShareHoldingExport.csv' file and insert into ShareHolding Table.
    """
    fb_name = fbname
    file_path = os.path.join(os.getcwd(), "FBFiles")

    fb_csv_path = os.path.join(file_path, fb_name) 
    fb_csv_file = os.path.join(fb_csv_path ,'ShareHolding.csv')
    
    file_to_check = file_path + fb_name + '\\' + 'ShareHolding.csv'
    
    if(fbname == 'intermediate_insert'):
        
        file_to_check = 'C:\\Users\\dsram\\BravisaLocalDeploy\\BravisaFiles\\MissingData'+ '\\' + 'ShareHolding.csv'
        fb_csv_file = file_to_check

    if os.path.isfile(file_to_check):
        try:
            table = pd.read_csv(fb_csv_file, engine='python')

            table['SHPDate'] = pd.to_datetime(table['SHPDate'], errors = 'ignore')
            table['SHPDate'] = table['SHPDate'].apply(lambda dt: dt.strftime('%Y-%m-%d')if not pd.isnull(dt) else '')

            table['ModifiedDate'] = pd.to_datetime(table['ModifiedDate'], errors = 'ignore')
            table['ModifiedDate'] = table['ModifiedDate'].apply(lambda dt: dt.strftime('%Y-%m-%d')if not pd.isnull(dt) else '')
            
            #Update Logic - Deletes based on key columns 
            print("Executing delete logic", flush = True)
            table_to_delete = table.groupby(['CompanyCode', 'SHPDate'], as_index=False).count()
            '''self.logfile.write("\tIn function: " + str(fb_csv_file.split("\\")[-1].split(".")[0]) + 
                        "\t\tTotal Loops:" + str(int(table_to_delete.shape[0])) + "\t\tFor file: " + str(fbname) + "\n")
            self.logfile.flush()'''
            for index,row in table_to_delete.iterrows():
                # self.logfile.write("\tIn function: " + str(fb_csv_file.split("\\")[-1].split(".")[0]) + 
                # 		"\t\tTotal Loops:" + str(int(table_to_delete.shape[0])) + "\t\tFor file: " + str(fbname) + "\n")
                # self.logfile.flush()
                sql = 'DELETE FROM public."ShareHolding" WHERE "CompanyCode" =\'' +str(row['CompanyCode']) + '\' AND  "SHPDate"=\'' + str(row['SHPDate']) + '\';'
                ##print(sql, flush = True)
                cur.execute(sql)
                conn.commit()
            

            print("Inserting Data into ShareHolding", flush = True)
            exportfilename = "ShareHoldingExport.csv"
            exportfile = open(exportfilename,"w",encoding='utf-8')
            table.to_csv(exportfile, header=True, index=False, lineterminator='\r')
            exportfile.close()
            
            copy_sql = """
                COPY "public"."ShareHolding" FROM stdin WITH CSV HEADER
                DELIMITER as ','
                """
            with open(exportfilename, 'r') as f:
                cur.copy_expert(sql=copy_sql, file=f)
                conn.commit()
                f.close()
            os.remove(exportfilename)
        except:
            print("Error in insert_shareholding()", flush = True)
            conn.rollback()

    else:
        print("File not found: " + fb_csv_file, flush = True)

def delete_OHLC(conn, cur, curr_date, end_date):
    try:
        ohlc_sql = f"""DELETE FROM public."OHLC" WHERE "Date" >= '{curr_date}' AND "Date" <= '{end_date}'"""
        cur.execute(ohlc_sql)
        conn.commit()
        print("Deleted OHLC data")
    except Exception as e:
        print(f"Error deleting OHLC data: {e}")

    try:
        nse_sql = f"""DELETE FROM public."NSE" WHERE "TIMESTAMP" >= '{curr_date}' AND "TIMESTAMP" <= '{end_date}'"""
        cur.execute(nse_sql)
        conn.commit()
        print("Deleted NSE data")
    except Exception as e:
        print(f"Error deleting NSE data: {e}")

    try:
        bse_sql = f"""DELETE FROM public."BSE" WHERE "TRADING_DATE" >= '{curr_date}' AND "TRADING_DATE" <= '{end_date}'"""
        cur.execute(bse_sql)
        conn.commit()
        print("Deleted BSE data")
    except Exception as e:
        print(f"Error deleting BSE data: {e}")

# Establish database connection
try:
    conn = psycopg2.connect(
        host="localhost",
        database="BravisaDB",
        user="postgres",
        password="edsols"
    )
    cur = conn.cursor()
    date_range = pd.date_range(start='2024-11-22',end='2024-11-26')
    start_date = date_range[0]
    end_date = date_range[-1]
    delete_OHLC(conn, cur, start_date, end_date)
    date_format = "%d%m%Y"
    current_date = start_date
    while current_date <= end_date:
        if current_date.weekday() == 0:  # Monday
            prev_date = current_date - timedelta(days=2)  # Saturday
        else:
            prev_date = current_date - timedelta(days=1)  # Previous day

        print(current_date)
        insert_shareholding(conn, cur, "FB" + prev_date.strftime(date_format) + "03")
        insert_shareholding(conn, cur, "FB" + current_date.strftime(date_format) + "01")
        insert_shareholding(conn, cur, "FB" + current_date.strftime(date_format) + "02")
        print("Inserted: shareholding for : ", current_date)
        try:
            # ohlc.main(current_date)
            ohlc_nse, ohlc_bse, ohlc_joined, ohlc_bg,  ohlc_full = ohlc.main(current_date)
            split_bonus.main(current_date)
            delIRS(current_date, conn, cur)    
            btt_list, ohlc_irs, btt_ohlc_merge, ohlc_change, ohlc_backgroundinfo_merge, industry_merge, industry_list_calc_free_float, \
            industry_list_ff_ohlc, master_list, sector_divisor, subsector_divisor, industry_divisor, sub_industry_divisor, sector_divisor_index,\
            subsector_divisor_index, industry_divisor_index, sub_industry_divisor_index, sector_index, subsector_index, industry_index,\
            sub_industry_index,irs_list_rank, irs_list, irs_pe, industry_list = IRS().gen_irs_current(current_date,conn, cur)
            
            
        except FileNotFoundError:
            print(" there is no file for the date: ", current_date)    
        current_date += timedelta(days=1)
except Exception as e:
    print(f"Error connecting to the database: {e}")
    import traceback; traceback.print_exc();
finally:
    if cur:
        cur.close()
    if conn:
        conn.close()



Deleted OHLC data
Deleted NSE data
Deleted BSE data
2024-11-22 00:00:00
File not found: e:\BravisaProject\app\FBFiles\FB2111202403\ShareHolding.csv
File not found: e:\BravisaProject\app\FBFiles\FB2211202401\ShareHolding.csv
File not found: e:\BravisaProject\app\FBFiles\FB2211202402\ShareHolding.csv
Inserted: shareholding for :  2024-11-22 00:00:00

		 OHLC Fetch Service Started..........



NSE Fetch invoked for date :  2024-11-22 00:00:00
nse file :  e:\BravisaProject\app\OHLCFiles\BhavCopy_NSE_CM_0_0_0_20241122_F_0000.csv


BSE Fetch invoked for date :  20241122
bse file :  e:\BravisaProject\app\OHLCFiles\BhavCopy_BSE_CM_0_0_0_20241122_F_0000.csv
Attempting to insert NSE data into DB
NSE Insert Completed
Attempting to insert BSE data into DB
BSE Insert Completed
nulls while merging 618
Total BTT records fetched: 1088
Total BTT records matching ISINs in ohlc_full: 1080
Total rows updated in btt_fix: 0
OHLC count with CompanyCode: 1902
Final length of ohlc_full: 4008
OHLC Insert Comple

In [ ]:
from Test_reports import ohlc
import psycopg2
import pandas as pd
import warnings
import traceback

def delete_OHLC(conn, cur, curr_date, end_date):
    try:
        ohlc_sql = f"""DELETE FROM public."OHLC" WHERE "Date" >= '{curr_date}' AND "Date" <= '{end_date}'"""
        cur.execute(ohlc_sql)
        conn.commit()
        print("Deleted OHLC data")
    except Exception as e:
        print(f"Error deleting OHLC data: {e}")

    try:
        nse_sql = f"""DELETE FROM public."NSE" WHERE "TIMESTAMP" >= '{curr_date}' AND "TIMESTAMP" <= '{end_date}'"""
        cur.execute(nse_sql)
        conn.commit()
        print("Deleted NSE data")
    except Exception as e:
        print(f"Error deleting NSE data: {e}")

    try:
        bse_sql = f"""DELETE FROM public."BSE" WHERE "TRADING_DATE" >= '{curr_date}' AND "TRADING_DATE" <= '{end_date}'"""
        cur.execute(bse_sql)
        conn.commit()
        print("Deleted BSE data")
    except Exception as e:
        print(f"Error deleting BSE data: {e}")
        
warnings.filterwarnings("ignore")
conn = psycopg2.connect(
    host="localhost",
    database="BravisaDB",
    user="postgres",
    password="edsols"
)

cur = conn.cursor()

date_range = pd.date_range(start='2024-01-05',end='2024-01-05')

try:
    delete_OHLC(conn, cur, date_range[0], date_range[-1])
    for date in date_range:
        ohlc_nse, ohlc_bse, ohlc_joined, ohlc_bg,  ohlc_full = ohlc.main(date) 
        ohlc_full.to_csv('ohlc_new.csv', index=False)
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
finally:
    if cur:
        cur.close()
    if conn:
        conn.close()
        


File Path : d:\BravisaProject\app\OHLCFiles\OHLCFiles\
Deleted OHLC data
Deleted NSE data
Deleted BSE data

		 OHLC Fetch Service Started..........



NSE Fetch invoked for date :  2024-01-05 00:00:00
nse file :  d:\BravisaProject\app\OHLCFiles\OHLCFiles\BhavCopy_NSE_CM_0_0_0_20240105_F_0000.csv


BSE Fetch invoked for date :  20240105
bse file :  d:\BravisaProject\app\OHLCFiles\OHLCFiles\BhavCopy_BSE_CM_0_0_0_20240105_F_0000.csv
Attempting to insert NSE data into DB
NSE Insert Completed
Attempting to insert BSE data into DB
BSE Insert Completed
nulls while merging 683
0       IN000958C024
1       IN0020060086
2       IN0020150101
3       IN0020160027
4       IN0020160043
            ...     
4502    INF846K01X63
4503    INF846K01Y96
4504    INF846K01Z04
4505    INF846K01Z12
4506    INF959L01HI3
Name: ISIN, Length: 683, dtype: object
Total BTT records fetched: 1002
Total BTT records matching ISINs in ohlc_full: 999
Total rows updated in btt_fix: 23
OHLC count with CompanyCode: 1783
Fin

In [ ]:
from Test_reports import ohlc
import psycopg2
import pandas as pd
import warnings
import traceback

def delete_OHLC(conn, cur, curr_date, end_date):
    try:
        ohlc_sql = f"""DELETE FROM public."OHLC" WHERE "Date" >= '{curr_date}' AND "Date" <= '{end_date}'"""
        cur.execute(ohlc_sql)
        conn.commit()
        print("Deleted OHLC data")
    except Exception as e:
        print(f"Error deleting OHLC data: {e}")

    try:
        nse_sql = f"""DELETE FROM public."NSE" WHERE "TIMESTAMP" >= '{curr_date}' AND "TIMESTAMP" <= '{end_date}'"""
        cur.execute(nse_sql)
        conn.commit()
        print("Deleted NSE data")
    except Exception as e:
        print(f"Error deleting NSE data: {e}")

    try:
        bse_sql = f"""DELETE FROM public."BSE" WHERE "TRADING_DATE" >= '{curr_date}' AND "TRADING_DATE" <= '{end_date}'"""
        cur.execute(bse_sql)
        conn.commit()
        print("Deleted BSE data")
    except Exception as e:
        print(f"Error deleting BSE data: {e}")
        
warnings.filterwarnings("ignore")
conn = psycopg2.connect(
    host="localhost",
    database="BravisaDB",
    user="postgres",
    password="edsols"
)

cur = conn.cursor()

date_range = pd.date_range(start='2024-01-05',end='2024-01-05')

try:
    delete_OHLC(conn, cur, date_range[0], date_range[-1])
    for date in date_range:
        ohlc_nse, ohlc_bse, ohlc_joined, ohlc_bg,  ohlc_full = ohlc.main(date) 
        ohlc_full.to_csv('ohlc_new.csv', index=False)
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
finally:
    if cur:
        cur.close()
    if conn:
        conn.close()
        


File Path : d:\BravisaProject\app\OHLCFiles\OHLCFiles\
Deleted OHLC data
Deleted NSE data
Deleted BSE data

		 OHLC Fetch Service Started..........



NSE Fetch invoked for date :  2024-01-05 00:00:00
nse file :  d:\BravisaProject\app\OHLCFiles\OHLCFiles\BhavCopy_NSE_CM_0_0_0_20240105_F_0000.csv


BSE Fetch invoked for date :  20240105
bse file :  d:\BravisaProject\app\OHLCFiles\OHLCFiles\BhavCopy_BSE_CM_0_0_0_20240105_F_0000.csv
Attempting to insert NSE data into DB
NSE Insert Completed
Attempting to insert BSE data into DB
BSE Insert Completed
nulls while merging 683
0       IN000958C024
1       IN0020060086
2       IN0020150101
3       IN0020160027
4       IN0020160043
            ...     
4502    INF846K01X63
4503    INF846K01Y96
4504    INF846K01Z04
4505    INF846K01Z12
4506    INF959L01HI3
Name: ISIN, Length: 683, dtype: object
Total BTT records fetched: 1002
Total BTT records matching ISINs in ohlc_full: 999
Total rows updated in btt_fix: 23
OHLC count with CompanyCode: 1783
Fin

In [ ]:
from Test_reports import ohlc
import psycopg2
import pandas as pd
import warnings
import traceback

def delete_OHLC(conn, cur, curr_date, end_date):
    try:
        ohlc_sql = f"""DELETE FROM public."OHLC" WHERE "Date" >= '{curr_date}' AND "Date" <= '{end_date}'"""
        cur.execute(ohlc_sql)
        conn.commit()
        print("Deleted OHLC data")
    except Exception as e:
        print(f"Error deleting OHLC data: {e}")

    try:
        nse_sql = f"""DELETE FROM public."NSE" WHERE "TIMESTAMP" >= '{curr_date}' AND "TIMESTAMP" <= '{end_date}'"""
        cur.execute(nse_sql)
        conn.commit()
        print("Deleted NSE data")
    except Exception as e:
        print(f"Error deleting NSE data: {e}")

    try:
        bse_sql = f"""DELETE FROM public."BSE" WHERE "TRADING_DATE" >= '{curr_date}' AND "TRADING_DATE" <= '{end_date}'"""
        cur.execute(bse_sql)
        conn.commit()
        print("Deleted BSE data")
    except Exception as e:
        print(f"Error deleting BSE data: {e}")
        
warnings.filterwarnings("ignore")
conn = psycopg2.connect(
    host="localhost",
    database="BravisaDB",
    user="postgres",
    password="edsols"
)

cur = conn.cursor()

date_range = pd.date_range(start='2024-01-05',end='2024-01-05')

try:
    delete_OHLC(conn, cur, date_range[0], date_range[-1])
    for date in date_range:
        ohlc_nse, ohlc_bse, ohlc_joined, ohlc_bg,  ohlc_full = ohlc.main(date) 
        ohlc_full.to_csv('ohlc_new.csv', index=False)
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
finally:
    if cur:
        cur.close()
    if conn:
        conn.close()
        


File Path : d:\BravisaProject\app\OHLCFiles\OHLCFiles\
Deleted OHLC data
Deleted NSE data
Deleted BSE data

		 OHLC Fetch Service Started..........



NSE Fetch invoked for date :  2024-01-05 00:00:00
nse file :  d:\BravisaProject\app\OHLCFiles\OHLCFiles\BhavCopy_NSE_CM_0_0_0_20240105_F_0000.csv


BSE Fetch invoked for date :  20240105
bse file :  d:\BravisaProject\app\OHLCFiles\OHLCFiles\BhavCopy_BSE_CM_0_0_0_20240105_F_0000.csv
Attempting to insert NSE data into DB
NSE Insert Completed
Attempting to insert BSE data into DB
BSE Insert Completed
nulls while merging 683
0       IN000958C024
1       IN0020060086
2       IN0020150101
3       IN0020160027
4       IN0020160043
            ...     
4502    INF846K01X63
4503    INF846K01Y96
4504    INF846K01Z04
4505    INF846K01Z12
4506    INF959L01HI3
Name: ISIN, Length: 683, dtype: object
Total BTT records fetched: 1002
Total BTT records matching ISINs in ohlc_full: 999
Total rows updated in btt_fix: 23
OHLC count with CompanyCode: 1783
Fin

In [ ]:
import psycopg2
import pandas as pd
import datetime

def delete_OHLC(conn, cur, curr_date, end_date):
    try:
        ohlc_sql = f"""DELETE FROM public."OHLC" WHERE "Date" >= '{curr_date}' AND "Date" <= '{end_date}'"""
        cur.execute(ohlc_sql)
        conn.commit()
        print("Deleted OHLC data")
    except Exception as e:
        print(f"Error deleting OHLC data: {e}")

    try:
        nse_sql = f"""DELETE FROM public."NSE" WHERE "TIMESTAMP" >= '{curr_date}' AND "TIMESTAMP" <= '{end_date}'"""
        cur.execute(nse_sql)
        conn.commit()
        print("Deleted NSE data")
    except Exception as e:
        print(f"Error deleting NSE data: {e}")

    try:
        bse_sql = f"""DELETE FROM public."BSE" WHERE "TRADING_DATE" >= '{curr_date}' AND "TRADING_DATE" <= '{end_date}'"""
        cur.execute(bse_sql)
        conn.commit()
        print("Deleted BSE data")
    except Exception as e:
        print(f"Error deleting BSE data: {e}")

# Establish database connection
try:
    conn = psycopg2.connect(
        host="localhost",
        database="BravisaDB",
        user="postgres",
        password="edsols"
    )
    cur = conn.cursor()
    delete_OHLC(conn, cur, '2024-01-04', '2024-01-04')
except Exception as e:
    print(f"Error connecting to the database: {e}")
finally:
    if cur:
        cur.close()
    if conn:
        conn.close()



In [1]:
from Test_reports import ohlc
import psycopg2
import pandas as pd
import warnings
import traceback

def delete_OHLC(conn, cur, curr_date, end_date):
    try:
        ohlc_sql = f"""DELETE FROM public."OHLC" WHERE "Date" >= '{curr_date}' AND "Date" <= '{end_date}'"""
        cur.execute(ohlc_sql)
        conn.commit()
        print("Deleted OHLC data")
    except Exception as e:
        print(f"Error deleting OHLC data: {e}")

    try:
        nse_sql = f"""DELETE FROM public."NSE" WHERE "TIMESTAMP" >= '{curr_date}' AND "TIMESTAMP" <= '{end_date}'"""
        cur.execute(nse_sql)
        conn.commit()
        print("Deleted NSE data")
    except Exception as e:
        print(f"Error deleting NSE data: {e}")

    try:
        bse_sql = f"""DELETE FROM public."BSE" WHERE "TRADING_DATE" >= '{curr_date}' AND "TRADING_DATE" <= '{end_date}'"""
        cur.execute(bse_sql)
        conn.commit()
        print("Deleted BSE data")
    except Exception as e:
        print(f"Error deleting BSE data: {e}")
        
warnings.filterwarnings("ignore")
conn = psycopg2.connect(
    host="localhost",
    database="BravisaDB",
    user="postgres",
    password="edsols"
)

cur = conn.cursor()

date_range = pd.date_range(start='2024-01-05',end='2024-01-05')

try:
    delete_OHLC(conn, cur, date_range[0], date_range[-1])
    for date in date_range:
        ohlc_nse, ohlc_bse, ohlc_joined, ohlc_bg,  ohlc_full = ohlc.main(date) 
        ohlc_full.to_csv('ohlc_new.csv', index=False)
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
finally:
    if cur:
        cur.close()
    if conn:
        conn.close()
        


File Path : d:\BravisaProject\app\OHLCFiles\OHLCFiles\
Deleted OHLC data
Deleted NSE data
Deleted BSE data

		 OHLC Fetch Service Started..........



NSE Fetch invoked for date :  2024-01-05 00:00:00
nse file :  d:\BravisaProject\app\OHLCFiles\OHLCFiles\BhavCopy_NSE_CM_0_0_0_20240105_F_0000.csv


BSE Fetch invoked for date :  20240105
bse file :  d:\BravisaProject\app\OHLCFiles\OHLCFiles\BhavCopy_BSE_CM_0_0_0_20240105_F_0000.csv
Attempting to insert NSE data into DB
NSE Insert Completed
Attempting to insert BSE data into DB
BSE Insert Completed
nulls while merging 683
0       IN000958C024
1       IN0020060086
2       IN0020150101
3       IN0020160027
4       IN0020160043
            ...     
4502    INF846K01X63
4503    INF846K01Y96
4504    INF846K01Z04
4505    INF846K01Z12
4506    INF959L01HI3
Name: ISIN, Length: 683, dtype: object
Total BTT records fetched: 1002
Total BTT records matching ISINs in ohlc_full: 999
Total rows updated in btt_fix: 23
OHLC count with CompanyCode: 1783
Fin

## Monthly missing BackgroundInfo

In [7]:
import psycopg2
import pandas as pd
import datetime

conn = psycopg2.connect(
    host="localhost",
    database="BravisaDB",
    user="postgres",
    password="edsols"
)

cur = conn.cursor()

current_date = '2024-01-04'
start_date__of_the_month = datetime.datetime.strptime(current_date, '%Y-%m-%d').replace(day=1).strftime('%Y-%m-%d')
end_date_of_the_month = pd.to_datetime(current_date).to_period('M').end_time.strftime('%Y-%m-%d')

print(start_date__of_the_month, end_date_of_the_month)

btt_list_sql = """SELECT * FROM public."BTTList" """
# where "BTTDate" >= '{}' and "BTTDate" <= '{}'""".format(start_date__of_the_month, end_date_of_the_month)
btt_list = pd.read_sql_query(btt_list_sql, conn)

background_info_sql = """SELECT * FROM public."BackgroundInfo" """
background_info = pd.read_sql_query(background_info_sql, conn)

#for distinct companycodes, check both tables have them else report the missing companycodes from backgroundinfo
missing_companycodes = []
for companycode in btt_list['CompanyCode'].unique():
    if companycode not in background_info['CompanyCode'].unique():
        missing_companycodes.append(companycode)
print(len(missing_companycodes))
print(missing_companycodes)

# for distinct ISIN codes in bttlist, check if they are present in backgroundinfo and report the missing ISIN codes
missing_isin = []
for isin in btt_list['ISIN'].unique():
    if isin not in background_info['ISINCode'].unique():
        missing_isin.append(isin)
print(len(missing_isin))
print(missing_isin)
print()

2024-01-01 2024-01-31
5
[11050093.0, 15620019.0, 16070141.0, 16590031.0, 17090024.0]
100
['34-TRIAL-INE451A01017 100', 'INE066A01013', '34-TRIAL-INE371A01025 100', 'INE545A01016', 'INE999A01015', 'INE765D01014', 'INE570A01014', 'INE802C01017', 'INE180C01026', 'INE927D01028', 'INE918I01018', 'INE466L01020', 'INE602A01023', 'INE597I01010', 'INE254N01018', 'INE982B01019', 'INE775B01025', 'INE036B01022', 'INE040A01026', 'INE858B01011', 'INE476A01014', 'INE239A01016', 'INE528G01027', 'INE00VM01010', 'INE200M01013', 'INE797F01012', 'INE121A01016', 'INE229C01013', 'INE239C01012', 'INE00WC01019', 'INE513A01014', 'INE962Y01013', 'INE821I01014', 'INE461D01010', 'INE116A01024', 'INE150B01021', 'INE410B01029', 'INE884A01019', 'INE863B01011', 'INE486A01013', 'INE035D01012', 'INE663A01017', 'INE183A01016', 'INE081A01012', 'INE335A01012', 'INE324A01024', 'INE451D01011', 'INE702C01019', 'INE385C01013', 'INE943C01027', 'INE825A01012', 'INE572A01028', 'INE064C01014', 'INE571A01020', 'INE930H01023', 'INE

In [5]:
import psycopg2
import pandas as pd
import datetime

conn = psycopg2.connect(
    host="localhost",
    database="BravisaDB",
    user="postgres",
    password="edsols"
)

cur = conn.cursor()

current_date = '2024-01-05'
start_date__of_the_month = datetime.datetime.strptime(current_date, '%Y-%m-%d').replace(day=1).strftime('%Y-%m-%d')
end_date_of_the_month = pd.to_datetime(current_date).to_period('M').end_time.strftime('%Y-%m-%d')

print(start_date__of_the_month, end_date_of_the_month)

btt_list_sql = """SELECT * FROM public."BTTList" """
                #   where "BTTDate" >= '{}' and "BTTDate" <= '{}'""".format(start_date__of_the_month, end_date_of_the_month)
btt_list = pd.read_sql_query(btt_list_sql, conn)

ShareHolding_sql = """SELECT * FROM public."ShareHolding" """ 
ShareHolding = pd.read_sql_query(ShareHolding_sql, conn)

# get missing companycodes that are present in bttlist but not in ShareHolding
missing_companycodes = []
for companycode in btt_list['CompanyCode'].unique():
    if companycode not in ShareHolding['CompanyCode'].unique():
        missing_companycodes.append(companycode)
print(len(missing_companycodes))
print(missing_companycodes)




2024-01-01 2024-01-31
13
[14094822.0, 14094977.0, 14094980.0, 17200004.0, 17200007.0, 17200009.0, 17200012.0, 14095554.0, 17200016.0, 17200019.0, 17200023.0, 15150124.0, 15210746.0]


In [15]:
# program to compamre two csvfiles
import pandas as pd
import datetime
from utils.db_helper import DB_Helper

conn = DB_Helper().db_connect()
cur = conn.cursor()

# '2024-01-05'
curr_date = datetime.datetime.strptime('2024-01-05', '%Y-%m-%d')

file1 = pd.read_csv('ohlc_new.csv')
file2 = pd.read_csv('old_ohlc.csv')

# sort them based on company code
file1 = file1.sort_values(by='CompanyCode')
file2 = file2.sort_values(by='CompanyCode')

# print null rows in file1 where the subset of colmns are CompanyCode, NSECode, BSECode, ISIN
print(len(file1[file1[['CompanyCode', 'NSECode', 'BSECode', 'ISIN']].isnull().any(axis=1)]))
print(len(file2[file2[['CompanyCode', 'NSECode', 'BSECode', 'ISIN']].isnull().any(axis=1)]))

BTT_back = datetime.date(curr_date.year, curr_date.month, 1).strftime("%Y-%m-%d")
next_month = curr_date.month + 1 if curr_date.month + 1 <= 12 else 1
next_year = curr_date.year if curr_date.month + 1 <= 12 else curr_date.year + 1
BTT_next = datetime.date(next_year, next_month, 1).strftime("%Y-%m-%d")

# Fetch BTT list data for the current month
btt_sql = """
SELECT *
FROM "BTTList"
WHERE "BTTDate" >= %s AND "BTTDate" < %s
"""
bttlist = pd.read_sql_query(btt_sql, con=conn, params=(BTT_back, BTT_next))
    
# print the missing company codes in file1 that are present in bttlist
missing_companycodes = []
for companycode in bttlist['CompanyCode'].unique():
    if companycode not in file1['CompanyCode'].unique():
        missing_companycodes.append(companycode)

print(len(missing_companycodes))
print(missing_companycodes)

2111
2128
2
[14094980.0, 17200012.0]


In [6]:
from utils.db_helper import DB_Helper
import pandas as pd
conn = DB_Helper().db_connect()
cur = conn.cursor()

# get shareholding columns
shareholding_sql = """SELECT * FROM public."ShareHolding" """
shareholding = pd.read_sql_query(shareholding_sql, conn)

print(shareholding.columns)

C:\Users\dsram\AppData\Local\Temp\ipykernel_6344\453760155.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  shareholding = pd.read_sql_query(shareholding_sql, conn)


Index(['CompanyCode', 'SHPDate', 'Capital', 'FaceValue', 'NoOfShares',
       'Promoters', 'Directors', 'SubsidiaryCompanies', 'OtherCompanies',
       'ICICI', 'UTI', 'IDBI', 'GenInsuranceComp', 'LifeInsuranceComp',
       'StateFinCorps', 'InduFinCorpIndia', 'ForeignNRI',
       'ForeignCollaborators', 'ForeignOCB', 'ForeignOthers',
       'ForeignInstitutions', 'ForeignIndustries', 'StateGovt', 'CentralGovt',
       'GovtCompanies', 'GovtOthers', 'Others', 'NBanksMutualFunds',
       'HoldingCompanies', 'GeneralPublic', 'Employees',
       'FinancialInstitutions', 'ForeignPromoter', 'GDR',
       'PersonActingInConcert', 'Total', 'ModifiedDate'],
      dtype='object')


In [10]:
import os
import subprocess
from utils.db_helper import DB_Helper

# Connect to the database
conn = DB_Helper().db_connect()
cur = conn.cursor()

# Create backup directory if it does not exist
backup_dir = 'backup'
if not os.path.exists(backup_dir):
    os.makedirs(backup_dir)

# Backup the database using pg_dump
backup_file = os.path.join(backup_dir, 'backup.sql')
pg_dump_command = f'pg_dump -U postgres -h localhost -p 5432 -d BravisaDB -F c -b -v -f {backup_file}'

# Set the PGPASSWORD environment variable for the PostgreSQL user password
os.environ['PGPASSWORD'] = 'edsols'

# Execute the pg_dump command and capture the output
process = subprocess.Popen(pg_dump_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()

# Check if the command was successful
if process.returncode == 0:
    print(f"Database backup successful. Backup file created at {backup_file}")
else:
    print(f"Database backup failed with exit code {process.returncode}")
    print(f"Error message: {stderr.decode()}")

# Clean up the environment variable
del os.environ['PGPASSWORD']


Database backup failed with exit code 1
Error message: 'pg_dump' is not recognized as an internal or external command,
operable program or batch file.



In [ ]:
import os
import subprocess
from utils.db_helper import DB_Helper

# Connect to the database
conn = DB_Helper().db_connect()
cur = conn.cursor()

# Create backup directory if it does not exist
backup_dir = 'backup'
if not os.path.exists(backup_dir):
    os.makedirs(backup_dir)

# Backup the database using pg_dump
backup_file = os.path.join(backup_dir, 'backup.sql')
pg_dump_command = f'pg_dump -U postgres -h localhost -p 5432 -d BravisaDB -F c -b -v -f {backup_file}'

# Set the PGPASSWORD environment variable for the PostgreSQL user password
os.environ['PGPASSWORD'] = 'edsols'

# Execute the pg_dump command and capture the output
process = subprocess.Popen(pg_dump_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()

# Check if the command was successful
if process.returncode == 0:
    print(f"Database backup successful. Backup file created at {backup_file}")
else:
    print(f"Database backup failed with exit code {process.returncode}")
    print(f"Error message: {stderr.decode()}")

# Clean up the environment variable
del os.environ['PGPASSWORD']


In [ ]:
# restore the database using pg_restore
restore_file = os.path.join(backup_dir, 'backup.sql')
pg_restore_command = f'pg_restore -U postgres -h localhost -p 5432 -d BravisaDB -c -v -F c -d {restore_file}'
os.environ['PGPASSWORD'] = 'edsols'

# Execute the pg_restore command and capture the output
process = subprocess.Popen(pg_restore_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()

# Check if the command was successful
if process.returncode == 0:
    print(f"Database restore successful. Database restored from {restore_file}")
else:
    print(f"Database restore failed with exit code {process.returncode}")
    print(f"Error message: {stderr.decode()}")
    
# Clean up the environment variable
del os.environ['PGPASSWORD']


In [9]:
from utils.db_helper import DB_Helper
import pandas as pd

db = DB_Helper()
conn = db.db_connect()

cur = conn.cursor()

last_insert_date_sql = """SELECT MAX("log_date") FROM "logs"."insert" """
last_report_generation_date_sql = """SELECT MAX("log_date") FROM "logs"."report_generation" """

last_insert_date = pd.read_sql_query(last_insert_date_sql, conn)
last_report_generation_date = pd.read_sql_query(last_report_generation_date_sql, conn)

print(last_insert_date.values[0][0])
print(last_report_generation_date.values[0][0])


2024-11-29
2024-11-28


C:\Users\dsram\AppData\Local\Temp\ipykernel_9196\1454160463.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  last_insert_date = pd.read_sql_query(last_insert_date_sql, conn)
C:\Users\dsram\AppData\Local\Temp\ipykernel_9196\1454160463.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  last_report_generation_date = pd.read_sql_query(last_report_generation_date_sql, conn)
